In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.



from IPython.display import display, Markdown, clear_output, HTML, Javascript, display_html
import subprocess
import sys

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.get_selected_index()+1)'))

HTML('''<script>{
    $('div.input').hide();
    }''')

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.get_selected_index()+1)'))
HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    The raw code for this IPython notebook is by default hidden for easier reading toggle on/off the raw code by clicking <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
import pandas as pd 
import numpy as np
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from itertools import combinations, permutations, product
from math import radians, cos, sin, asin, sqrt
from ast import literal_eval as make_tuple
from datetime import datetime, timedelta,date, time
import os
import random
import pulp
import csv
from io import StringIO
import scipy as sc
from scipy import optimize as op
import json
import time
import logging
import sys
from collections import deque
import re


def toggle_on_click_mode(change):
    if change.new != None:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


def toggle_on_start_mode(change):
    if change.new != None:
        clear_output()
        display(widgets.VBox([mode_title,mode_btns]))
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

        
stndmode_btn = widgets.ToggleButtons(
    options=['Standard'],
    value =None,
    disabled=False,
    button_style='info'
)        
        
        

mode_btns = widgets.ToggleButtons(
    options=['Standard', 'Fixed Spaces'],
    value =None,
    disabled=False,
    button_style='info'
)

start_title = widgets.Output()
with start_title:
    display(Markdown("# Generate Floorplan"))
    display(Markdown("This notebook is the final step of the Workspace Collaboration Optimizer. Select standard mode to create an optimized seating plan using the previously validated input files in the Final Files folder. After creating a seating plan in standard mode you will also have the option to run Fixed Spaces mode. Utilize Fixed Spaces mode after editing your original seating plan. This option will place unassigned teams in workspaces based on your validated files, while keeping your changes to the seatplan. Please see the documentation for more detailed steps on using this feature."))
    display(Markdown("##  Click below to start standard mode"))


mode_title = widgets.Output()
with mode_title:
    display(Markdown("##  Fixed Space option added"))
    display(Markdown("<b> To run Fixed Spaces, complete standard mode steps below to create your optimized floorplan. Then make any necessary changes to your floorplan and save it as a csv. Circle back to this notebook and click Fixed Spaces button below:</b> "))
    
    
# standard_btn = widgets.Button(description='Standard',button_style = 'primary')

# fixed_spcs_btn = widgets.Button(description='Fixed Spaces',button_style = 'primary')

# variable_seats_btn = widgets.Button(description='Variable Seats',button_style = 'primary')

clear_output()
#display(Markdown("##  Choose your mode"))
display(widgets.VBox([start_title,stndmode_btn]))

stndmode_btn.observe(toggle_on_start_mode,'value')

mode_btns.observe(toggle_on_click_mode,'value')



In [ ]:
def go_to_nextblock(btn):
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+3)'))

global fixedSpaces
#global variableSeats
global mode
clear_output()

cwd = os.getcwd()

load_fsp_file = FileChooser(cwd)
load_tsz_file = FileChooser(cwd)
load_spc_file = FileChooser(cwd)

style = {'description_width': '135px'}
layout = {'width': '385px'}
create_outputfile_Name = widgets.Output()
with create_outputfile_Name:
    display(Markdown("#### Give the name of the output file here without a .xlsx extension at the end. The output file will be saved in the app_results folder in the same location as this python file"))

name_out_file = widgets.Text(placeholder='No.xlsx extension at the end',
        description='New results name:',
        layout=layout,
        style=style,
        disabled=False
        )

out = widgets.Output(layout={'border': '1px solid black'})
CGREEN  = '\33[32m'
CRED = '\033[91m'
CEND = '\033[0m'
BOLD = '\033[1m'

go_to_next_btn = widgets.Button(description='Go',button_style = 'primary')
go_to_next_btn.on_click(go_to_nextblock)

if mode_btns.value == None:
    mode_btns.value = 'Standard'
    
    ###############################################################
    #############IF STANDARD IS CHOSEN:######################\

if mode_btns.value == 'Standard':
    
    display(Markdown("## Standard"))

    display(Markdown("<b>All four files will be programmatically read from previous stage of validations from the Final Files set folder:</b> "))
    display(Markdown(("{interactions.csv, team_size.csv, space_capacity.csv, and distance.csv}")))



    display(out)        

    #display(Markdown("#### Give the name of the output file here without a .xlsx extension at the end. The output file will be saved in the app_results folder in the same location as this python file"))
    

    
    



    cwd = os.getcwd()
    inputpathffs = cwd + '\\Final Files set\\'        
    input_path = inputpathffs

    fixedSpaces = False
    variableSeats = False
    
    display(create_outputfile_Name)
    display(widgets.HBox([name_out_file,go_to_next_btn]))

    mode = 0
    
    fp_orig = None
    fp_modif = None
    
    ###############################################################
    #############IF FIXED SPACES IS CHOSEN:######################
    
    
    
elif mode_btns.value == 'Fixed Spaces':
    
    
    
    #load_fsp_file = FileChooser(cwd)
    clear_output()
    loadfiles_markdown = widgets.Output()
    tab_nest_fsp = widgets.Accordion()
    
    
    
    display(Markdown("## Fixed Spaces"))

    with loadfiles_markdown:
        display(Markdown('<div class="alert alert-block alert-info"><b> Load Data: </b> Select your fixed space file.</div>'))

   

    display(widgets.VBox([create_outputfile_Name,name_out_file]))

    
    display(Markdown("#### For fixed spaces, select the location of the initial floor plan with teams fixed to assigned spaces as well as the name of the file."))
    display(Markdown("#### The fixed space plan file should have the exact same format as the floorplan that is output by the code without fixed spaces.  It should also be in .csv format."))
    display(Markdown(" Example: C:/myfolder/files/fs_floorplan.csv"))
    
    fixedSpaces = True
    variableSeats = False
    
    tab_nest_fsp.children = [load_fsp_file]
    tab_nest_fsp.set_title(0,'Fixed Spaces file')
    display(loadfiles_markdown)
    display(widgets.VBox([tab_nest_fsp,go_to_next_btn]))
    display(out)
   
    #display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

 
    
    fxspcs_mkdown = widgets.Output()
    fxspcs_mkdown2 = widgets.Output()

    outreq = widgets.Output(layout={'border': '2px solid blue'})

    tab_nest_fxspcs = widgets.Accordion(selected_index=None)
    style = {'description_width': '185px'}
    layout = {'width': '505px'}




    with fxspcs_mkdown:
        display(Markdown('<div class="alert alert-block alert-info"><b>  Validate Fixed Spaces File:</div>'))
    #display(teamsize_mkdown)

    with fxspcs_mkdown2:

        display(Markdown('<p>&nbsp; &nbsp; a. <b>Validate space size comparison</b>.</p>'))
        display(Markdown('<p>&emsp; &emsp; i. Ensure that the sum of all rows for each column (which corresponds to spaces) in the user provided fixed space file does not exceed the space capacity of that space.</p>'))
        display(Markdown('<p>&nbsp; &nbsp; b. <b>Validate dimensions layout</b>.</p>'))
    #     display(Markdown('<p>&emsp; &emsp; i. No NA, None or string</p>'))
        display(Markdown('<p>&emsp; &emsp; i. Ensure that the size of the fixed space file is the same dimensions as the floor plan that will be produced by the run.  So make sure the file is has n rows and k columns, where n is the number of groups and k is  the number of spaces. </p>'))


    tab_nest_fxspcs.children = [widgets.VBox([fxspcs_mkdown,fxspcs_mkdown2])]

    tab_nest_fxspcs.set_title(0,'Validate Fixed Spaces File Requirements')
    
    mode = 2
    
    
    
    
mode_map = {
  
  0: {'mode': 'Standard',
     'reDistAdjSeats': False, 'officeOutput': False, 'relConstraints':False, 'relConstraintsForm': False,
     'numTries': 1, 'flatInteractionBool': True, 'isRerun': False, 'officeFloorCapacityFileFormat': True, 
     'scaleUnits': False, 'relContraints': False, 'fpRank': False,
     'runFWOptimizer': True, 'runScipyTrOptimizer': False, 
     'preSetupTeamAllocation': list(), 'changeIdx': None, 'rawCsvs': dict(), 
     'modechangeIdx': None, 'validation': True, 'pseudoTeams':False},
  1: {'mode': 'Interactive',
     'reDistAdjSeats': False, 'officeOutput': False, 'relConstraints': False, 'relConstraintsForm': False,
     'numTries': 1, 'flatInteractionBool': True, 'isRerun': True, 'officeFloorCapacityFileFormat': True, 
     'scaleUnits': False, 'relContraints': False, 'fpRank': False,
     'runFWOptimizer': True, 'runScipyTrOptimizer': False, 
     'preSetupTeamAllocation': list(), 'changeIdx': None, 'rawCsvs': dict(), 
     'modechangeIdx': None, 'validation': False, 'pseudoTeams':False},
  2: {'mode': 'Fixed Spaces',
     'reDistAdjSeats': False, 'officeOutput': False, 'relConstraints': False, 'relConstraintsForm': False,
     'numTries': 1, 'flatInteractionBool': True, 'isRerun': True, 'officeFloorCapacityFileFormat': True, 
     'scaleUnits': False, 'relContraints': False, 'fpRank': False,
     'runFWOptimizer': True, 'runScipyTrOptimizer': False, 
     'preSetupTeamAllocation': list(), 'changeIdx': None, 'rawCsvs': dict(), 
     'modechangeIdx': None, 'validation': True, 'pseudoTeams':False},
  3: {'mode': 'CoLocated Teams',
     'reDistAdjSeats': False, 'officeOutput': False, 'relConstraints':False, 'relConstraintsForm': False,
     'numTries': 1, 'flatInteractionBool': True, 'isRerun': False, 'officeFloorCapacityFileFormat': True, 
     'scaleUnits': False, 'relContraints': False, 'fpRank': False,
     'runFWOptimizer': True, 'runScipyTrOptimizer': False, 
     'preSetupTeamAllocation': list(), 'changeIdx': None, 'rawCsvs': dict(), 
     'modechangeIdx': None, 'validation': True, 'pseudoTeams':True}
}
  
mode_params = mode_map[int(mode)]
notebook_mode = mode_params['mode']
#print(f"MODE: {notebook_mode}")    






#############################################################
#  Helper routines - Prepare data for Frank Wolfe algo
#############################################################

def rawFCtoPandas(rawFC):
    """
    transposes a matrix. (12 x 2) --> (1 x 13)


    inputs:
    ------
    rawFC: df; floor capacity
    
    returns:
    -------
    newFC: df; floor capacity transposed
    """
    try:
        newFC = rawFC.transpose().reset_index()
        newFC.columns = newFC.iloc[0,:]
        newFC = newFC.drop([0], axis = 0).reset_index(drop = True)
        cols = newFC.columns
        newFC[cols[1]] = pd.to_numeric(newFC[cols[1]])
        return newFC
    except: 
        return {"error":"error within [rawFCtoPandas]"}
    #newFC['Capacity'] = pd.to_numeric(newFC['Capacity'])
    

def getFloorNamesDF(floor_capacity):
    """
    drops duplicates from floor_capacity

    inputs:
    -------
    floor_capacity

    returns:
    -----
    floor_capacity
    """
    cols = floor_capacity.columns
    return floor_capacity[cols[0]].drop_duplicates().reset_index(drop = True)
    #return floor_capacity[["Floor"]].drop_duplicates().reset_index(drop = True)

def getTeamNamesDF(team_size):
    """
    drops duplicates from team_size
    """
    cols = team_size.columns
    return team_size[[cols[0]]].drop_duplicates().reset_index(drop = True)
    #return team_size[["Team"]].drop_duplicates().reset_index(drop = True)

def validateFloors(floorNames, rawDistanceCols, numFloors):
    """
    used for validation of floor names.
    """
    #this can be a placeholder for future string name validations
    #currently just the length
    return (len(floorNames) == len(rawDistanceCols)) and (len(floorNames) == numFloors)

def validateTeams(teamNames, rawInteractionsCols, numTeams):
    """
    used for validation of teamNames
    """
    #this can be a placeholder for future string name validations
    #currently just the length
    return (len(teamNames) == len(rawInteractionsCols)) and (len(teamNames) == numTeams) 

def getTeamSizeVector(team_size):
    #TO DO. Deprecate in next version
    cols = team_size.columns
    return tuple(team_size[cols[1]].values)
    #return tuple(team_size["Size"].values)

def getTeamSizeVector2(team_size):
    # please note this version now assumes team names are on Index --> so Team Size is read from FIRST column
    cols = team_size.columns
    return tuple(team_size[cols[0]].values)

def sortFloorsDF(fcDF, dropIdx = True):
    """
    sorts the floor columns by alphabetical + numerical order.
    this is important to make sure data looks consistent throughout the process.

    ex. [NORTH-FLOOR-1, NORTH-FLOOR-2, EAST-FLOOR-1] --> [EAST-FLOOR-1, NORTH-FLOOR-1, NORTH-FLOOR-2]

    inputs:
    ------
    fcDF: pandas dataframe also known as rawfloorCapacity .
    """
    cols = fcDF.columns
    sortedDF = fcDF.sort_values(by = [cols[0]])
    if (dropIdx):
        sortedDF = sortedDF.reset_index(drop = True)
    else:
        sortedDF = sortedDF.reset_index()
    return sortedDF

def sortTeamsDF(tsDF, dropIdx = True):
    """
    columns names can be changed. thats why we take it by relative position
    
    sorts the team column by alphabetical + numerical order.
    this is important to make sure data looks consistent throughout the process.

    ex. [SALES, ENGINEERING, MARKETING] --> [ENGINEERING, MARKETING, SALES]

    inputs:
    ------
    tsDF: pandas dataframe also known as team size
    
    """
    cols = tsDF.columns
    sortedDF = tsDF.sort_values(by = [cols[0]])
    if (dropIdx):
        sortedDF = sortedDF.reset_index(drop = True)
    else:
        sortedDF = sortedDF.reset_index()
    return sortedDF

def sortInteractions(iDF):
    rawCols = iDF.columns
    idxMap = {} 
    #sort rows
    sortedRawI = iDF.sort_values([rawCols[0]])
    #keep old rows idx i.e. just put it aside and forget about it (kept copy for audit purposes)
    copyIdxRows = sortedRawI.reset_index().iloc[:, [0,1]]
    copyIdxRows.rename(columns = {"index" : "orig_index"}, inplace = True)
    idxMap["rows"] = copyIdxRows.copy()
    sortedRawI = sortedRawI.reset_index(drop = True)
    #transpose 
    temp = sortedRawI.set_index(sortedRawI.columns[0])
    transposeRawI = temp.transpose()
    transposeRawI = transposeRawI.reset_index()
    transposeRawI.rename(columns = {"index" : rawCols[0]}, inplace = True)
    transposeRawI.columns.name = None
    #sort columns 
    sortedRawI2 = transposeRawI.sort_values([transposeRawI.columns[0]])
    #record old column idx, put it aside and forget about it (just in case it is needed... kept a copy...)
    copyIdxCols = sortedRawI2.reset_index().iloc[:, [0,1]]
    copyIdxCols.rename(columns = {"index" : "orig_index"}, inplace = True)
    idxMap["cols"] = copyIdxCols.copy()
    sortedRawI2 = sortedRawI2.reset_index(drop = True)
    #transpose back
    temp2 = sortedRawI2.set_index(sortedRawI2.columns[0])
    transposeRawI2 = temp2.transpose()
    transposeRawI2 = transposeRawI2.reset_index()
    transposeRawI2.rename(columns = {"index" : rawCols[0]}, inplace = True)
    transposeRawI2.columns.name = None
    return transposeRawI2, idxMap 

def sortDistance(tDF):
    rawCols = tDF.columns
    idxMap = {} 
    #sort rows
    sortedRawT = tDF.sort_values([rawCols[0]])
    #record old rows idx, put it aside and forget about it (just in case it is needed... kept a copy...)
    copyIdxRows = sortedRawT.reset_index().iloc[:, [0,1]]
    copyIdxRows.rename(columns = {"index" : "orig_index"}, inplace = True)
    idxMap["rows"] = copyIdxRows.copy()
    sortedRawT = sortedRawT.reset_index(drop = True)
    #transpose 
    temp = sortedRawT.set_index(sortedRawT.columns[0])
    transposeRawT = temp.transpose()
    transposeRawT = transposeRawT.reset_index()
    transposeRawT.rename(columns = {"index" : rawCols[0]}, inplace = True)
    transposeRawT.columns.name = None
    #sort columns 
    sortedRawT2 = transposeRawT.sort_values([transposeRawT.columns[0]])
    #record old column idx, put it aside and forget about it (just in case it is needed... kept a copy...)
    copyIdxCols = sortedRawT2.reset_index().iloc[:, [0,1]]
    copyIdxCols.rename(columns = {"index" : "orig_index"}, inplace = True)
    idxMap["cols"] = copyIdxCols.copy()
    sortedRawT2 = sortedRawT2.reset_index(drop = True)
    temp2 = sortedRawT2.set_index(sortedRawT2.columns[0])
    transposeRawT2 = temp2.transpose()
    transposeRawT2 = transposeRawT2.reset_index()
    transposeRawT2.rename(columns = {"index" : rawCols[0]}, inplace = True)
    transposeRawT2.columns.name = None
    return transposeRawT2, idxMap

def sortFloorplan(floorplan):
    """
    sorts floorplan columns and rows. makes index team name
    
    inputs:
    --------
    floorplan: floorplan dataframe - either original or interactive
    
    returns:
    -----
    transposeRawFP2: sorted floor plan
    idxMap: we don't use that..
    """
    if isinstance(floorplan, pd.DataFrame): 
        rawCols = floorplan.columns
        idxMap = {} 
        #sort rows
        sortedFP = floorplan.sort_values([rawCols[0]])
        #keep old rows idx i.e. just put it aside and forget about it (kept copy for audit purposes)
        copyIdxRows = sortedFP.reset_index().iloc[:, [0,1]]
        copyIdxRows.rename(columns = {"index" : "orig_index"}, inplace = True)
        idxMap["rows"] = copyIdxRows.copy()
        sortedFP = sortedFP.reset_index(drop = True)
        #transpose 
        temp = sortedFP.set_index(sortedFP.columns[0])
        transposeRawFP = temp.transpose()
        transposeRawFP = transposeRawFP.reset_index()
        transposeRawFP.rename(columns = {"index" : rawCols[0]}, inplace = True)
        transposeRawFP.columns.name = None
        #sort columns 
        sortedRawFP2 = transposeRawFP.sort_values([transposeRawFP.columns[0]])
        #record old column idx, put it aside and forget about it (just in case it is needed... kept a copy...)
        copyIdxCols = sortedRawFP2.reset_index().iloc[:, [0,1]]
        copyIdxCols.rename(columns = {"index" : "orig_index"}, inplace = True)
        idxMap["cols"] = copyIdxCols.copy()
        sortedRawFP2 = sortedRawFP2.reset_index(drop = True)
        #transpose back
        temp2 = sortedRawFP2.set_index(sortedRawFP2.columns[0])
        transposeRawFP2 = temp2.transpose()
        transposeRawFP2 = transposeRawFP2.reset_index()
        transposeRawFP2.rename(columns = {"index" : rawCols[0]}, inplace = True)
        transposeRawFP2.columns.name = None

        transposeRawFP2.index = transposeRawFP2[rawCols[0]]
        transposeRawFP2 = transposeRawFP2.drop(columns=[rawCols[0]])
        return transposeRawFP2, idxMap 
    else:
        return None, None

#############################################################
#  Helper routines - Scale data for FW Algo
#############################################################
def minMaxScaler(x, exclude_cols = []):
    if isinstance(x,np.ndarray):
        z = x.copy()
    elif isinstance(x, pd.DataFrame):
        #keep cols order (whatever it may be)
        origCols = tuple(x.columns)
        tempColsAll = [ str(s) + "_"+ str(i) for i,s in enumerate(x.columns)]
        tempColsMapIdxAll = { s : i for i,s in enumerate(tempColsAll)}
        temp1 = x.copy()
        temp1.columns = tempColsAll
        #keep numeric columns
        temp2 = temp1.select_dtypes(include = 'number')
        numCols = temp2.columns
        z = temp2.to_numpy()
                
    min_x = z.min()
    z -= min_x
    minMaxRange = z.ptp()
    
    if minMaxRange != 0:
        z = z / minMaxRange
        if isinstance(x,np.ndarray):
            return z, minMaxRange, min_x
        elif isinstance(x, pd.DataFrame):           
            for j in range(z.shape[1]):
                tempColsLabel = numCols[j] 
                cols_idx = tempColsMapIdxAll[tempColsLabel]
                temp1.iloc[:,cols_idx]= z[:,j]
                temp1.columns = origCols
            return temp1, minMaxRange, min_x
    else:
        return x, None, None

def minMaxUnScale(z, MinMaxRange, min_x):
    if MinMaxRange:
        if isinstance(z,np.ndarray):
            x = z.copy()
        elif isinstance(z, pd.DataFrame):
            #keep cols order (whatever it may be)
            origCols = tuple(z.columns)
            tempColsAll = [ str(s) + "_"+ str(i) for i,s in enumerate(z.columns)]
            tempColsMapIdxAll = { s : i for i,s in enumerate(tempColsAll)}
            temp1 = z.copy()
            temp1.columns = tempColsAll
            #keep numeric columns
            temp2 = temp1.select_dtypes(include = 'number')
            numCols = temp2.columns
            x = temp2.to_numpy()
        
        
        x = x * MinMaxRange + min_x
        
        if isinstance(z,np.ndarray):
            return x
        elif isinstance(z, pd.DataFrame):           
            for j in range(x.shape[1]):
                tempColsLabel = numCols[j] 
                cols_idx = tempColsMapIdxAll[tempColsLabel]
                temp1.iloc[:,cols_idx]= x[:,j]
                temp1.columns = origCols
            return temp1
    else:
        return z

#############################################################
#  Interactive Helper Functions
#############################################################
def cleanNanCopy(X):
    """
    replaces nulls with 0
    """
    if isinstance(X, pd.DataFrame): 
        temp = X.fillna(0)
    else: 
        temp = X.copy()
        temp[np.isnan(temp)] = 0
    return temp  

def getPreSetupTeamAllocation(X0, X1, teamSizeVec = None, byFloor = True ):
    #Assumes X0 and X1 are comming from user input 
    #and have actual team e.g 70 headcount instead of 0.6 team ratio
    if (X0.shape == X1.shape):
        res, keys = [],[]
        K,N = X0.shape[0], X0.shape[1]
        if teamSizeVec is not None:
            temp1 = np.round(cleanNanCopy(X0))
            temp2 = np.round(cleanNanCopy(X1))
            diff = temp2 - temp1
            keys =  np.where(diff!= 0)
            teamSizeMat = np.repeat(teamSizeVec,N).reshape(K,N)
            Xteamratio = np.round( X1 / teamSizeMat,4)
            if isinstance(Xteamratio, pd.DataFrame):
                res = [(x,y,Xteamratio.iloc[x,y]) for x,y in keys]
            else:
                res = [(x,y,Xteamratio[x,y]) for x,y in keys]
        else:
            temp1 = np.round(cleanNanCopy(X0))
            temp2 = np.round(cleanNanCopy(X1))
            diff = temp2 - temp1
            keys =  np.where(diff!= 0)
            if (byFloor):
                teams = set(keys[0])
                floorIdx = [ (i,j) for i in teams for j in range(N)]
                if isinstance(X1, pd.DataFrame):
                    res = [(x,y,temp2.iloc[x,y]) for x,y in floorIdx]
                else:
                    res = [(x,y,temp2[x,y]) for x,y in floorIdx]   
            else:
                if isinstance(X1, pd.DataFrame):
                    res = [(x,y,X1.iloc[x,y]) for x,y in zip(keys[0],keys[1]) ]
                else:
                    res = [(x,y,X1[x,y]) for x,y in zip(keys[0],keys[1])]
        return res, keys
        
def whereChangeFlat(X0,X1, K, N):
    if (X0.shape == X1.shape and X0.shape[0] == K and X0.shape[1] == N):
        temp1 = np.round(cleanNanCopy(X0))
        temp2 = np.round(cleanNanCopy(X1))
        # transpose and flatten
        flat1 = np.asarray(temp1).T.ravel()
        flat2 = np.asarray(temp2).T.ravel()
        diff = flat2 - flat1
        flatIdxs = np.where(diff !=0)
        return flatIdxs[0]

def isValidpreSetupAllocation(preSetupTeamAllocation, teamSizeVector ):
    teamSize = {}
    for x,y,v in preSetupTeamAllocation:
        if x not in teamSize:
            teamSize[x] = v
        else:
            teamSize[x] = v + teamSize[x]
    for i,v in teamSize.items():
        if (teamSize[i] != teamSizeVector[i]):
            return False
    return True

def getX1(input_path = ""):
    X = pd.read_csv(input_path+"floorplacment000.csv")
    return X

def getX2(input_path = ""):
    modifiedX = pd.read_csv(input_path+"floorplacment.csv")
    return modifiedX

def getChangeVectors(X1,X2):
    """
    this is a key function in Interactive Mode and Fixed Spaces. it identifies the changes a user made.

    inputs:
    ------
    X1: original floor plan
    X2: modified floor plan

    returns:
    ------
    preSet: known as preSetupTeamAllocation in code; has format (team_indx, floor_indx, value) --> (0, 1, 71)
        --> (engineering team, floor 1, +71 new heads)
    changeIdxs: also known as changeIdx; has format (team_indx, floor_indx); shows which cells are different
        between original and modified floor plan. Used by FWOptimizer
    """
    # if X1 is None: getX1() # pandas dataframe 
    # if X2 is None: getX2() 
    preSetup, Idxs = getPreSetupTeamAllocation(X1,X2, byFloor=False)
    print(preSetup)
    changeIdxs = whereChangeFlat(X1,X2, X1.shape[0], X1.shape[1])
    return preSetup, changeIdxs


def createInteractionsMatrix2(flatInteractions):
    """
    if flatInteractionBool is True we run this function. we turn a big dataframe with 
    daily interactions into a smaller, summed one.

    inputs:
    -------
    flatInteractions: also known as rawI or rawIraw in code. it is a pandas dataframe which has
        the collab time between team 1 (TimeInvestors) and team 2 (Collaborators) on a date.
    
    returns:
    -------
    df: also known as rawI. it sums of the total time spent in collaboration between teams.
        if flatInteractions had 5 unique teams, it sum up how much time the 5 teams have spent working
        with the other 5 teams.


    sums of total time spent in collaboration.
    """
    #get unique names of participants
    organizersNames = flatInteractions.iloc[:,0].unique()
    contributorsNames = flatInteractions.iloc[:,1].unique()
    allNames = set(organizersNames).union(set(contributorsNames))
    removeNames = ["Other_Collaborators", "Unclassified_Internal", "Unclassified_External", "Collaborators Within Group"]
    keepNames = allNames - set(removeNames)
    cols = flatInteractions.columns
    
    #add up withing group collaboration 
    temp = flatInteractions.copy()
    temp.loc[temp[cols[1]]=='Collaborators Within Group', cols[1]] = temp[cols[0]]
    temp = temp.drop(temp.columns[[2]], axis=1) # drop date column
    
    #remove unwanted rows
    temp =  temp[~ ((temp[cols[0]].isin(list(removeNames))) | (temp[cols[1]].isin(list(removeNames))))]
    
    #build matrix
    key_cols = [cols[0], cols[1]]
    interactionsMatrix = temp.set_index(key_cols).groupby(key_cols)[cols[-1]].agg(['sum']).unstack()


    #clean and wrap up
    df = interactionsMatrix.fillna(0.0)
    df.columns = df.columns.droplevel(0)
    df.columns.name = None
    df.index.name = None
    df = df.reset_index()
    df.rename(columns = {df.columns[0]: 'Collab' }, inplace = True)
    return df

#############################################################
#  Helper routines - Load relative constraints
#############################################################

def validTeamNames(teamNames, sRawTS):
    names = set(teamNames)
    validNames = set(sRawTS[sRawTS.columns[0]])
    return names.issubset(validNames)

def getTeamIdsDF(sRawTS):
    return (sRawTS[[sRawTS.columns[0]]].reset_index())[[sRawTS.columns[0],"index"]]

def getFloorIdsDF(floorsDF):
    return (floorsDF[[floorsDF.columns[0]]].reset_index())[[floorsDF.columns[0],"index"]]

def parseConstraints(rawCons, sRawTS):
    rawCons.rename(columns = {rawCons.columns[0]: "Constraint_Type",\
                          rawCons.columns[1]: "From",\
                          rawCons.columns[2]: "To",\
                          rawCons.columns[3]: "Eq_Type",\
                          rawCons.columns[4]: "Amount"\
                          }, inplace = True)

    rawCons["From"] = rawCons["From"].str.strip()
    rawCons["To"] = rawCons["To"].str.strip()
    allNames = set(rawCons["From"]).union(set(rawCons["To"]))
    if validTeamNames(allNames, sRawTS):
        rawCons["Constraint_Type"] = rawCons["Constraint_Type"].str.upper()
        dcDF = rawCons[rawCons["Constraint_Type"] == "DISTANCE"]
        dcDF = dcDF.sort_values(by = ['From','To'])
        icDF = rawCons[rawCons["Constraint_Type"] == "INTERACTION"]
        return dcDF, icDF
    else:
        print("error relative constraints team names not valid")
        return None,None
   

In [ ]:
clear_output()
global out_filename
if len(name_out_file.value)> 0:
    out_filename = name_out_file.value+".xlsx"
    print(BOLD,out_filename)
    display(out)

else:
    timestamp = datetime.now().replace(microsecond=0).isoformat()
    timestamp = timestamp.replace(":","")
    print('You did not enter an output file name above. A default name of [floor_plan_' + timestamp+ ' ] will be used.')
    out_filename = 'floor_plan_'+ timestamp +'.xlsx'
    print(BOLD,out_filename)
    display(out)
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+1)'))


In [ ]:
# for local files
clear_output()

writer = None
base_path = './app_results/' 

#print(os.getcwd())
#print(base_path)



if not (os.path.exists(base_path) and os.path.isdir(base_path)):
        os.mkdir(base_path)
        print('app_results folder has been created')
else:
    print('app_results folder already exists')
        
#check dir is created
#all_dirs = [ x[0] for x in os.walk(".")]
#all_dirs

#helper function to save results
class write2File:
    def __init__(self, writer = None, output = './app_results/output.xlsx'):
        if not writer:
            self.writer = pd.ExcelWriter(output,  engine ='xlsxwriter')
        else:
            self.writer = writer
        self.output = output   
      
    def to_excel(self,df, tab_name = "001"):
         df.to_excel(self.writer, tab_name)
    
    def save(self):
        self.writer.save()



###########################################################################
######## Helper routine for FW algo - mostly unchanged from Jupyter #######
###########################################################################



#############################################################
#  Helper routines - Prepare data for Frank Wolfe algo
#############################################################

def rawFCtoPandas(rawFC):
    """
    transposes a matrix. (12 x 2) --> (1 x 13)


    inputs:
    ------
    rawFC: df; floor capacity
    
    returns:
    -------
    newFC: df; floor capacity transposed
    """
    try:
        newFC = rawFC.transpose().reset_index()
        newFC.columns = newFC.iloc[0,:]
        newFC = newFC.drop([0], axis = 0).reset_index(drop = True)
        cols = newFC.columns
        newFC[cols[1]] = pd.to_numeric(newFC[cols[1]])
        return newFC
    except: 
        return {"error":"error within [rawFCtoPandas]"}
    #newFC['Capacity'] = pd.to_numeric(newFC['Capacity'])
    

def getFloorNamesDF(floor_capacity):
    """
    drops duplicates from floor_capacity

    inputs:
    -------
    floor_capacity

    returns:
    -----
    floor_capacity
    """
    cols = floor_capacity.columns
    return floor_capacity[cols[0]].drop_duplicates().reset_index(drop = True)
    #return floor_capacity[["Floor"]].drop_duplicates().reset_index(drop = True)

def getTeamNamesDF(team_size):
    """
    drops duplicates from team_size
    """
    cols = team_size.columns
    return team_size[[cols[0]]].drop_duplicates().reset_index(drop = True)
    #return team_size[["Team"]].drop_duplicates().reset_index(drop = True)

def validateFloors(floorNames, rawDistanceCols, numFloors):
    """
    used for validation of floor names.
    """
    #this can be a placeholder for future string name validations
    #currently just the length
    return (len(floorNames) == len(rawDistanceCols)) and (len(floorNames) == numFloors)

def validateTeams(teamNames, rawInteractionsCols, numTeams):
    """
    used for validation of teamNames
    """
    #this can be a placeholder for future string name validations
    #currently just the length
    return (len(teamNames) == len(rawInteractionsCols)) and (len(teamNames) == numTeams) 

def getTeamSizeVector(team_size):
    #TO DO. Deprecate in next version
    cols = team_size.columns
    return tuple(team_size[cols[1]].values)
    #return tuple(team_size["Size"].values)

def getTeamSizeVector2(team_size):
  # please note this version now assumes team names are on Index --> so Team Size is read from FIRST column
    cols = team_size.columns
    return tuple(team_size[cols[0]].values)

def sortFloorsDF(fcDF, dropIdx = True):
    """
    sorts the floor columns by alphabetical + numerical order.
    this is important to make sure data looks consistent throughout the process.

    ex. [NORTH-FLOOR-1, NORTH-FLOOR-2, EAST-FLOOR-1] --> [EAST-FLOOR-1, NORTH-FLOOR-1, NORTH-FLOOR-2]

    inputs:
    ------
    fcDF: pandas dataframe also known as rawfloorCapacity .
    """
    cols = fcDF.columns
    sortedDF = fcDF.sort_values(by = [cols[0]])
    if (dropIdx):
        sortedDF = sortedDF.reset_index(drop = True)
    else:
        sortedDF = sortedDF.reset_index()
    return sortedDF

def sortTeamsDF(tsDF, dropIdx = True):
    """
    columns names can be changed. thats why we take it by relative position
    
    sorts the team column by alphabetical + numerical order.
    this is important to make sure data looks consistent throughout the process.

    ex. [SALES, ENGINEERING, MARKETING] --> [ENGINEERING, MARKETING, SALES]

    inputs:
    ------
    tsDF: pandas dataframe also known as team size
    
    """
    cols = tsDF.columns
    sortedDF = tsDF.sort_values(by = [cols[0]])
    if (dropIdx):
        sortedDF = sortedDF.reset_index(drop = True)
    else:
        sortedDF = sortedDF.reset_index()
    return sortedDF

def sortInteractions(iDF):
    rawCols = iDF.columns
    idxMap = {} 
    #sort rows
    sortedRawI = iDF.sort_values([rawCols[0]])
    #keep old rows idx i.e. just put it aside and forget about it (kept copy for audit purposes)
    copyIdxRows = sortedRawI.reset_index().iloc[:, [0,1]]
    copyIdxRows.rename(columns = {"index" : "orig_index"}, inplace = True)
    idxMap["rows"] = copyIdxRows.copy()
    sortedRawI = sortedRawI.reset_index(drop = True)
    #transpose 
    temp = sortedRawI.set_index(sortedRawI.columns[0])
    transposeRawI = temp.transpose()
    transposeRawI = transposeRawI.reset_index()
    transposeRawI.rename(columns = {"index" : rawCols[0]}, inplace = True)
    transposeRawI.columns.name = None
    #sort columns 
    sortedRawI2 = transposeRawI.sort_values([transposeRawI.columns[0]])
    #record old column idx, put it aside and forget about it (just in case it is needed... kept a copy...)
    copyIdxCols = sortedRawI2.reset_index().iloc[:, [0,1]]
    copyIdxCols.rename(columns = {"index" : "orig_index"}, inplace = True)
    idxMap["cols"] = copyIdxCols.copy()
    sortedRawI2 = sortedRawI2.reset_index(drop = True)
    #transpose back
    temp2 = sortedRawI2.set_index(sortedRawI2.columns[0])
    transposeRawI2 = temp2.transpose()
    transposeRawI2 = transposeRawI2.reset_index()
    transposeRawI2.rename(columns = {"index" : rawCols[0]}, inplace = True)
    transposeRawI2.columns.name = None
    return transposeRawI2, idxMap 

def sortDistance(tDF):
    rawCols = tDF.columns
    idxMap = {} 
    #sort rows
    sortedRawT = tDF.sort_values([rawCols[0]])
    #record old rows idx, put it aside and forget about it (just in case it is needed... kept a copy...)
    copyIdxRows = sortedRawT.reset_index().iloc[:, [0,1]]
    copyIdxRows.rename(columns = {"index" : "orig_index"}, inplace = True)
    idxMap["rows"] = copyIdxRows.copy()
    sortedRawT = sortedRawT.reset_index(drop = True)
    #transpose 
    temp = sortedRawT.set_index(sortedRawT.columns[0])
    transposeRawT = temp.transpose()
    transposeRawT = transposeRawT.reset_index()
    transposeRawT.rename(columns = {"index" : rawCols[0]}, inplace = True)
    transposeRawT.columns.name = None
    #sort columns 
    sortedRawT2 = transposeRawT.sort_values([transposeRawT.columns[0]])
    #record old column idx, put it aside and forget about it (just in case it is needed... kept a copy...)
    copyIdxCols = sortedRawT2.reset_index().iloc[:, [0,1]]
    copyIdxCols.rename(columns = {"index" : "orig_index"}, inplace = True)
    idxMap["cols"] = copyIdxCols.copy()
    sortedRawT2 = sortedRawT2.reset_index(drop = True)
    temp2 = sortedRawT2.set_index(sortedRawT2.columns[0])
    transposeRawT2 = temp2.transpose()
    transposeRawT2 = transposeRawT2.reset_index()
    transposeRawT2.rename(columns = {"index" : rawCols[0]}, inplace = True)
    transposeRawT2.columns.name = None
    return transposeRawT2, idxMap

def sortFloorplan(floorplan):
    """
    sorts floorplan columns and rows. makes index team name
    
    inputs:
    --------
    floorplan: floorplan dataframe - either original or interactive
    
    returns:
    -----
    transposeRawFP2: sorted floor plan
    idxMap: we don't use that..
    """
    if isinstance(floorplan, pd.DataFrame): 
        rawCols = floorplan.columns
        idxMap = {} 
        #sort rows
        sortedFP = floorplan.sort_values([rawCols[0]])
        #keep old rows idx i.e. just put it aside and forget about it (kept copy for audit purposes)
        copyIdxRows = sortedFP.reset_index().iloc[:, [0,1]]
        copyIdxRows.rename(columns = {"index" : "orig_index"}, inplace = True)
        idxMap["rows"] = copyIdxRows.copy()
        sortedFP = sortedFP.reset_index(drop = True)
        #transpose 
        temp = sortedFP.set_index(sortedFP.columns[0])
        transposeRawFP = temp.transpose()
        transposeRawFP = transposeRawFP.reset_index()
        transposeRawFP.rename(columns = {"index" : rawCols[0]}, inplace = True)
        transposeRawFP.columns.name = None
        #sort columns 
        sortedRawFP2 = transposeRawFP.sort_values([transposeRawFP.columns[0]])
        #record old column idx, put it aside and forget about it (just in case it is needed... kept a copy...)
        copyIdxCols = sortedRawFP2.reset_index().iloc[:, [0,1]]
        copyIdxCols.rename(columns = {"index" : "orig_index"}, inplace = True)
        idxMap["cols"] = copyIdxCols.copy()
        sortedRawFP2 = sortedRawFP2.reset_index(drop = True)
        #transpose back
        temp2 = sortedRawFP2.set_index(sortedRawFP2.columns[0])
        transposeRawFP2 = temp2.transpose()
        transposeRawFP2 = transposeRawFP2.reset_index()
        transposeRawFP2.rename(columns = {"index" : rawCols[0]}, inplace = True)
        transposeRawFP2.columns.name = None

        transposeRawFP2.index = transposeRawFP2[rawCols[0]]
        transposeRawFP2 = transposeRawFP2.drop(columns=[rawCols[0]])
        return transposeRawFP2, idxMap 
    else:
        return None, None

#############################################################
#  Helper routines - Scale data for FW Algo
#############################################################
def minMaxScaler(x, exclude_cols = []):
    if isinstance(x,np.ndarray):
        z = x.copy()
    elif isinstance(x, pd.DataFrame):
        #keep cols order (whatever it may be)
        origCols = tuple(x.columns)
        tempColsAll = [ str(s) + "_"+ str(i) for i,s in enumerate(x.columns)]
        tempColsMapIdxAll = { s : i for i,s in enumerate(tempColsAll)}
        temp1 = x.copy()
        temp1.columns = tempColsAll
        #keep numeric columns
        temp2 = temp1.select_dtypes(include = 'number')
        numCols = temp2.columns
        z = temp2.to_numpy()
                
    min_x = z.min()
    z -= min_x
    minMaxRange = z.ptp()
    
    if minMaxRange != 0:
        z = z / minMaxRange
        if isinstance(x,np.ndarray):
            return z, minMaxRange, min_x
        elif isinstance(x, pd.DataFrame):           
            for j in range(z.shape[1]):
                tempColsLabel = numCols[j] 
                cols_idx = tempColsMapIdxAll[tempColsLabel]
                temp1.iloc[:,cols_idx]= z[:,j]
                temp1.columns = origCols
            return temp1, minMaxRange, min_x
    else:
        return x, None, None

def minMaxUnScale(z, MinMaxRange, min_x):
    if MinMaxRange:
        if isinstance(z,np.ndarray):
            x = z.copy()
        elif isinstance(z, pd.DataFrame):
            #keep cols order (whatever it may be)
            origCols = tuple(z.columns)
            tempColsAll = [ str(s) + "_"+ str(i) for i,s in enumerate(z.columns)]
            tempColsMapIdxAll = { s : i for i,s in enumerate(tempColsAll)}
            temp1 = z.copy()
            temp1.columns = tempColsAll
            #keep numeric columns
            temp2 = temp1.select_dtypes(include = 'number')
            numCols = temp2.columns
            x = temp2.to_numpy()
        
        
        x = x * MinMaxRange + min_x
        
        if isinstance(z,np.ndarray):
            return x
        elif isinstance(z, pd.DataFrame):           
            for j in range(x.shape[1]):
                tempColsLabel = numCols[j] 
                cols_idx = tempColsMapIdxAll[tempColsLabel]
                temp1.iloc[:,cols_idx]= x[:,j]
                temp1.columns = origCols
            return temp1
    else:
        return z

###########################################################################
######## Core Frank Wolfe Algo  #################
###########################################################################


def objectiveFunc(I,X,T):
    return np.trace(T @ X.T @ I @ X)
       #def ObjectiveFunc():
       #    X = np.asmatrix([ [X] * N for _ in range(K)])
       #    ObjFun = ((T % t(X) % I % X).transpose()).trace()
       #    return ObjFun

def boostTeamsInternalCollab(I, booster = 100):
    if (type(I) == np.matrix and I.shape[0] == I.shape[1]):
        boostI = I.copy()
        np.fill_diagonal(boostI, booster*np.max(I))
        return boostI


def FwOptimizer2(I,X,T,teamSizeVector, floorCapVector, K, N, changeIdx, relConstraints = False, teamIdMap = None, dcDF = None):
    """
    Frank Wolfe algorithm
    NOTE: Microsoft research paper
    # SEARCH FOR THIS TAG

    inputs:
    ------
    I: interactions matrix
    X: floor plan matrix
    T: distance matrix
    teamSizeVector: team size file
    floorCapVector: floor capacity file
    K: number of teams
    N: number of floors
    changeIdx: for interactive mode - delta of user changes - what user changes
    in interaction mode
    relConstraints: additional constraints that a user adds (boolean) 
    teamIdMap: when user uploads the file for the first time, it sorts the file and maps it to an id
    dcDF: relative constraints input from user for distance constraints
    
    returns:
    ------
    X: floor plan - matrix 
    success: boolean
    count: 
    txx: loss function to minmize
    """
    #initialize linear search
    try:
        eps = 0.001
        frobeniusNorm = eps+1
        count = 0
        gamma = 1.0 / (count + 2)
        debug = False
        pacifier = True
        verbose = False # turn on for debuging
        success = False
        initX = X
        while(frobeniusNorm > eps):
            ############################################################################################
            ## Data for Linear Program (formerly LpSove)
            ############################################################################################
            #Objective function. Min linear G(Xk)* Y 
            def getGradient(I, X, T):
              """
              previous1: #return (I + I.T) @ X @ T
              previous2: I.T @ X @ T + I @ X @ T.T --> (I^T X * T + I * X* T^T)
              """
              return I @ X @ T + I.T @ X @ T.T
            
                
      
            Gmat= getGradient(I, X, T)
            G =  np.array(Gmat.T).ravel()
            #Constraint 1. Allocate All team members i.e. sum of team = 100% of team size (acroos all floors)
            teamIdxPerFloor = []
            for start_i in range(K):
                temp_i = [K*j+start_i for j in range(N)]
                teamIdxPerFloor.append(temp_i)
            #Constraint 2. All variables >= 0
            # x_i allocation >= 0 (see linear program below) 
            #Constraint 3. Floor <= floor capacity
            floorIdxAllTeams = []
            for j in range(0,N*K,K):
                temp_j = [j + i for i in range(K)]
                floorIdxAllTeams.append(temp_j)
            #Constraint 4. Interactive 
            #   changeIdxs = whereChangeFlat(X1,X2, K, N)  #Calculated in driver program
            if(debug):    
                print(" G(Xk) : " , G)
                print("Constraint1.X.indexes: ", teamIdxPerFloor)
                print("Constraint3.X.indexes: ", floorIdxAllTeams)
            ############################################################################################
            ## Linear Program 
            ############################################################################################
            #Define problem
            prob = pulp.LpProblem("FW line search", pulp.LpMinimize )
            #Define variables
            var_idxs = list(range(N*K))
            S = pulp.LpVariable.dict("x_", var_idxs, lowBound=0, upBound = 1)
            #check_idxs(X, var_idxs)
            #Define objective Function.
            prob += sum([G[i] * S[i] for i in range(K*N)])
            #Constraint 1. Allocate all teams
            for i in range(K):
                prob += sum([S[j] for j in teamIdxPerFloor[i]]) == 1
            #Constraint 2. All variables >= 0
            for x_i in range(K*N):
                prob += S[x_i] >= 0
            #Constraint 3. floor <= floor capacity
            teamSize = np.tile(teamSizeVector,N)
            for j in range(N):
                 prob +=sum([teamSize[i] * S[i] for i in floorIdxAllTeams[j]])<= floorCapVector[j]
            #Constraint 4. Interactive
            if (changeIdx is not None and len(changeIdx) > 0):
                flatX = np.asarray(X).T.ravel()
                for i in changeIdx:
                    prob += S[i] == flatX[i]
            if(debug):
                print("  lpVariable.dict: ",  S)
                print("  lpProblem...",prob)
            #Constraint 5. Relative Constraint
            isRelCons = (relConstraints and (teamIdMap is not None))
            if(isRelCons):
                prob = addRelConstraintDistanceAvg2Avg(dcDF, T, S, teamIdMap, teamIdxPerFloor, prob)
                print("LpProb after adding constraints...")
                print(prob)
            ############################################################################################
            ## Lp Solve & Wrap up results
            ############################################################################################
            print("Solving... ")
            prob.solve()
            print("Status:", pulp.LpStatus[prob.status])
            Xid = { str(v) : k for k,v in S.items()}
            SoptVector = np.zeros(K*N)
            for v in prob.variables():
                #print(Xid[v.name], v.name, "=", v.varValue )
                SoptVector[int(Xid[v.name])] = v.varValue
            Sopt = np.matrix(SoptVector.reshape(N,K).T)
            if(debug):
                print("result: ", type(Sopt), np.round(Sopt,2))

            ########################################################################################
            ### Relative Constraints - Raw Output from Optimizer (for User Testing and Wire-framing)
            #########################################################################################
            #TO DO. Add addtional relative constraint test cases
            if(isRelCons and pulp.LpStatus[prob.status] == 'Optimal'):
                success = True
                return Sopt, success, count

            ############################################################################################
            ## Optimize step size (gamma)
            ############################################################################################
            #plugin line search result (1-gamma) Y - gamma X into objective function
            Y = Sopt
            tXX = np.trace(T @ X.T @ I @ X)
            tXY = np.trace(T @ X.T @ I @ Y) + np.trace(T @ Y.T @ I @ X)   
            tYY = np.trace(T @ Y.T @ I @ Y)
            a1 = tXX-tXY+tYY # gradient - how big of a jump to take
            b1 = tXY-2*tXX
            c1 = tXX
            #solve for gamma and optimize inside region of interest
            if(a1 > 0):
                #find minimum of quadratic a1*gamma^2 + b1*gamma + c1
                z = (-1*b1)/ (2*a1)    
                #by definition gamma [0,1] (gamma is weighted average of line between X and Y points in solution space)
                if (z>1): gamma = 1
                elif (z<0): gamma = 3/(count+2)
                else: gamma = z
            if(a1<0):
                #find maximum of quadratic a1*gamma^2 + b1*gamma + c1 
                z = (-1*b1)/ (2*a1)
                #by definition gamma [0,1] 
                if(z>=1): gamma = 3/(count+2)       # follow inverse direction of steepest descent
                elif(z<0): gamma = 1
                else:                     # inside polygon of valid solution
                    if((a1+b1)> 0): gamma = 3/(count+2)
                    else: gamma = 1
            if (a1==0):     
                #find minimum of linear b1*gamma  + c1 sbt [0,1] region
                if (b1 >=0): gamma = 3/(count+2)
                else: gamma = 1
            

            #update Xkplus1
            Xk = X
            X = Xk + gamma * (Y -Xk)
            if(debug):
                print("tXX :", round(tXX,4)) # what we're trying to minmize (a1)
                print("tXY :", round(tXY,4))
                print("tYY :", round(tYY,4))
                print("a1  : ", round(a1,2))
                print("b1  : ", round(b1,2))
                print("c1  : ", round(c1,2))
                print("z   : ",  (-1*b1)/ (2*a1))
                print("gamma: ", gamma)
            ############################################################################################
            ## Stop criteria
            ############################################################################################
            frobeniusNorm = np.linalg.norm(X-Xk, 'fro')
            count+=1
            #emergency break in case not converging
            #usualy converge after 15 iterations
            if (count>1000):
              print("more than 1000 iterations... cannot converge")
              break
            if(pacifier):
                print("lp iteration: " + str(count) + " fNorm : " + str(round(frobeniusNorm,6)))
        if(verbose):
                print("Initial placement... X0")
                print(initX)
                print("FW Optimal placement... X")
                print(X)
                print("Last Lp placement... Y")
                print(Y)
        if (frobeniusNorm <= eps):
            success = True
            return X, success, count, tXX
        else:
            success = False
            return X, success, count, tXX
    except:
        return {"error":"error within FwOptimizer"}



    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
###########################################################################
######## Initial placement heuristic - use arrangeTeams2  #################
###########################################################################


debug_temp = []
def arrangeTeams(iterFloor, iterTeam, I, T, c, s, flag_max, preSetupTeamAllocation, useTeamRatio = False): ##TODO: REVIEW
    """
      follows wpa collab data to set starting points. takes one team - looks for max collab in group-to-group query and place it close to the team it collabs closely with. (X0); starting point for frank wolfe algo
    """
    global debug_temp
    debug = False
    if isinstance(s,np.ndarray):
        requestedSeats = s.copy()
    elif isinstance(s, tuple) or isinstance(s, list):
        requestedSeats = np.array(s)
    #flag_max = FALSE
    
    N = T.shape[0]
    #K - teams number
    K = I.shape[0]

    listTeams = None
    #find the largest team
    teamInd = 0 #iterTeam  #which.max(s) #TODO: Update
    floorNum = iterFloor
     
    if listTeams == None:
        listTeams = [0]#[teamInd] 
    else:
        listTeams = [listTeams, teamInd]
  
    #################################################################################################
    #Changing capacity and team size according to user input/constrains for correct X0 initialization
    #################################################################################################
    X0 = np.asmatrix([ [0] * N for _ in range(K)])
    # i = 1
    persistantS = tuple(s)
    if (len(preSetupTeamAllocation) > 0):  
        for i in range(0, len(preSetupTeamAllocation)):
            constrain = preSetupTeamAllocation[i]
            indTeam = constrain[0]
            indFloor  = constrain[1]
            if useTeamRatio:
                teamRatio = constrain[2]
                teamAlloc  =  teamRatio * persistantS[indTeam]
            else:
                teamAlloc  = constrain[2]
                
            if(isinstance(X0,pd.DataFrame)):
                X0.iloc[indTeam, indFloor] = np.round(teamAlloc)
                requestedSeats[indTeam] = requestedSeats[indTeam] - np.round(teamAlloc)
            else:
                X0[indTeam, indFloor] = np.round(teamAlloc)
                requestedSeats[indTeam] = requestedSeats[indTeam] - np.round(teamAlloc)
            
    s = np.round(s, 4)
    c = np.round(c, 4)
    
    #inmutable copy of user requested initializations (to be used by placeTeamOnFloor)
    preAllocX =  X0.copy()
    preAllocX.setflags(write = 0)
           
    while (len(listTeams) <= K):
        if(debug):
            debug_temp.append((listTeams.copy(), teamInd, floorNum, requestedSeats.copy(), X0.copy()))
        res = placeTeamOnFloor(floorNum, teamInd, c, requestedSeats, X0, 1, preAllocX)
        X0 = res["X0"]
        
        floorNum = res["floorNum"]
        
        if (len(listTeams) == K):
            break

        #TODO: update names change froim length
        length = len(listTeams)
        pickFrom = list(set(list(range(0, len(s)))) - set(list(range(teamInd, teamInd+1))))
        while len(listTeams) != (length + 1):
            #TODO; dupplicate casting to list
            topCollabTeamInd = list([pickFrom[np.argmax(np.array(I[teamInd:teamInd+1,[pickFrom]].tolist()))]]) #TODO: CHECK FOR VALIDITY
            if flag_max:
                topCollabTeamInd = random.randint(0, len(delete))

            topCollabTeamInd = topCollabTeamInd[0]

            if (len(set(listTeams).intersection([topCollabTeamInd])) == 0):
                listTeams.append(topCollabTeamInd)

            pickFrom = list(set(pickFrom) - set([topCollabTeamInd])) #TODO REVIEW: changes to return List, set failing on place team on floor else cases
            
        teamInd = listTeams[len(listTeams)-1] #TODO: Review, added -1

    #rmatS = np.asmatrix([persistantS] * N)
    #X0 =  np.around(X0/np.rot90(rmatS, 3), 4)

    return X0


def arrangeTeams2(iterFloor, iterTeam, I, T, K, N, floorCapVector, teamSizeVector, flag_max, preSetupTeamAllocation, useTeamRatio = False, flatX0 = False):
    """
    flatX0 is now set to True
    """
    #Random initializatin. 
    #N - number of floors, K - number of teams
    res1 = []
    for floorID in range(N):
        if not flatX0: #not guaranteed to make constraint (equal number of team - can be greater or less)
#             print('ACTIVATED!')
            floorX0Ratio = 1/N*np.random.uniform(0,1,(1,K)) #same seed, same results; different seed, different results --> return best floor plan
            floorCap = [floorCapVector[floorID]] * K
            floorX0 = [ x * y for x, y in zip(floorCap, floorX0Ratio)] 
        else:
            floorX0Ratio = np.array([1/N] * K).reshape(1,K) # runs this when flatX0 is at default True
            floorCap = [floorCapVector[floorID]] * K
            floorX0 = [ x * y for x, y in zip(floorCap, floorX0Ratio)]
        res1.append(list(floorX0))
    res2 = np.concatenate(res1).T
    
    #### Add user input from "Interactive" mode. i.e. preSetupTeamAllocation ###
    X0 = res2
    if (len(preSetupTeamAllocation) > 0):  
        for i in range(0, len(preSetupTeamAllocation)):
            constrain = preSetupTeamAllocation[i]
            indTeam = constrain[0]
            indFloor  = constrain[1]
            if useTeamRatio:
                teamRatio = constrain[2]
                teamAlloc  =  teamRatio * persistantS[indTeam]
            else:
                teamAlloc  = constrain[2]
                
            if(isinstance(X0,pd.DataFrame)):
                X0.iloc[indTeam, indFloor] = np.round(teamAlloc)
                #requestedSeats[indTeam] = requestedSeats[indTeam] - np.round(teamAlloc)
            else:
                X0[indTeam, indFloor] = np.round(teamAlloc)
                #requestedSeats[indTeam] = requestedSeats[indTeam] - np.round(teamAlloc)
    
    return X0
  
  
  
  
 #s1 =[]

def placeTeamOnFloor(floorNum, teamInd, c, s, X0, folds, preAllocMat): ##TODO: COMPLETE
    s1 = s #.copy()
    if s1[teamInd] == 0:
        res = {}
        res["X0"] = X0
        res["s"] = s
        res["floorNum"] = floorNum
        return res

    #think on bad scenario
    #print(folds)
    if (folds  > len(c)):
        res = {}
        res["X0"] = -1
        res["s"] = -1 #updated team size
        res["floorNum"] = -1
        return res
   
    if (preAllocMat[teamInd, floorNum] == 0):
        
        occupiedSpaceOnFloor = sum(X0[:,floorNum])
        availableSpace = np.asmatrix(c[floorNum]) - occupiedSpaceOnFloor
        seatsNeededForTeam = s1[teamInd]

        if (availableSpace >= seatsNeededForTeam):
            X0[teamInd, floorNum] = seatsNeededForTeam      
            s1[teamInd] = 0

        else:
            s1[teamInd] = seatsNeededForTeam - availableSpace
            X0[teamInd, floorNum] = availableSpace

            if (floorNum < len(c)):
                floorNum += 1
            else:
                floorNum = 1

            folds += 1
    else:
        print("Skiping pre-allocated values.." , teamInd, floorNum, X0[teamInd,floorNum] )
        if (floorNum < len(c)):
            floorNum += 1
        else:
            floorNum = 1
        folds += 1

    return placeTeamOnFloor(floorNum, teamInd, c, s1, X0, folds,preAllocMat)



display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
######              START: local files INPUT 

##################################################
#       Load local Files ** Start Here **
###################################################
inputpathffs = cwd + '\\Final Files set\\'        
input_path = inputpathffs

if fixedSpaces:
    fs_input_path = load_fsp_file.selected_path + '\\'+ load_fsp_file.selected_filename
    

#parameters for X0 heuristics test
X0_heuristics_options = { 0 : "Azt Classic - WPA Data", 2: "Stacker Tool - Uniform", 3:  "Azt New - Larger  v1" }
X0_heuristics = X0_heuristics_options[2]


#input flat version of G2G query
flatInteractionBool = True

rawIraw = pd.read_csv(input_path+"interactions.csv")
if(flatInteractionBool):
    rawI = createInteractionsMatrix2(rawIraw)
rawT =  pd.read_csv(input_path+"distance.csv")


if not variableSeats:
    rawfloorCapacity = pd.read_csv(input_path+"space_capacity.csv")
    rawTS = pd.read_csv(input_path+"team_size.csv")
    
    col1_TS = rawTS.columns[0]
    rawTS[col1_TS] = rawTS[col1_TS].astype(str)
    

    

##################################################
#       Simulate Fixed Spaces INPUT  ** Here **
###################################################
if fixedSpaces:
    
    rawFixedSpaces = pd.read_csv(fs_input_path)

    #clean and prep fs file input
    rawFixedSpaces.fillna(0, inplace = True)
    fs_cols = rawFixedSpaces.columns
    rawFixedSpaces = rawFixedSpaces.sort_values(by = [fs_cols[0]])
    fs_team_names = list(rawFixedSpaces.iloc[:,0])
    rawFixedSpaces.index = fs_team_names 
    rawFixedSpaces.drop(columns = [fs_cols[0]], inplace = True)
    fs_space_names = list(rawFixedSpaces.columns)

    K = rawFixedSpaces.shape[0]
    N = rawFixedSpaces.shape[1]

    print("loading fixed spaces from file... ")
    print("fixed spaces input path:", load_fsp_file.selected_path)
    print("fixed spaces filename:", load_fsp_file.selected_filename)
    
    
    ### Use fixed spaces floorplan to create a floorplan that resembles the users floorplan with all zeroes as original_floorplan
    fp_orig = rawFixedSpaces.copy()
    fp_modif = rawFixedSpaces.copy()
    
    for col in fp_orig .columns:
        fp_orig[col].values[:] = 0
    #############################################################################
    ## FIXED SPACES VALIDATIONS:
    #1
        ##Validation #1.  Ensure that the sum of all rows for each column (which corresponds to spaces) 
        ##in the user provided fixed space file does not exceed the space capacity of that space.

    getfloorcaps = rawFixedSpaces.reset_index()
    getfloorcaps = getfloorcaps.drop(getfloorcaps.columns[0],axis=1)
    floorcaps1 = getfloorcaps.melt(var_name='Floor',value_name='Total Used').groupby('Floor').sum()
    floorcaps1 = floorcaps1.reset_index()
    floorcaps_validation1 = floorcaps1.merge(rawfloorCapacity,on='Floor')
    floorcaps_validation1['exceedsCapacity'] = np.select([floorcaps_validation1['Total Used'] > floorcaps_validation1['Total Capacity']],[1],default=0)
    validation1_fsp_failed = floorcaps_validation1[floorcaps_validation1['exceedsCapacity']==1]   
    if len(validation1_fsp_failed) > 0:
        print(CRED,BOLD,'Validation Fixed Spaces 1 Error:',CEND,' One or more rows in the user provided fixed space file', BOLD,'exceeds',CEND,'the space capacity of that space')
        display(out)
        display(validation1_fsp_failed)
        raise SystemExit('Validation Fixed Spaces 1 Error!')
    else:
        print(CGREEN,'Validation Fixed Spaces 1-Passed')



    ##2
        ##Validation #2.  Ensure that the size of the fixed space file is the same dimensions as the floor plan that 
        ##will be produced by the run.  So make sure the file is has n rows and k columns, 
        ##where n is the number of groups and k is  the number of spaces.

    dimensions_INTX = set(rawIraw[rawIraw.columns[0]])
    dimensions_FC = set(rawfloorCapacity[rawfloorCapacity.columns[0]])

    indx_rawFSP = rawFixedSpaces.reset_index().copy()
    dimensions_row_FSP = set(indx_rawFSP[indx_rawFSP.columns[0]])

    colscount_FSP= len(rawFixedSpaces.columns)
    dimensions_col_FSP = set(getfloorcaps[getfloorcaps.columns[0:colscount_FSP]])

    if(len(dimensions_col_FSP) > len(dimensions_FC)):
        uniquediffFSP = dimensions_col_FSP - dimensions_FC
    else:
        uniquediffFSP = dimensions_FC - dimensions_col_FSP

    if(len(dimensions_row_FSP) > len(dimensions_INTX)):
        uniquediffFSP2 = dimensions_row_FSP - dimensions_INTX
    else:
        uniquediffFSP2 = dimensions_INTX - dimensions_row_FSP

    if len(uniquediffFSP) > 0 and len(uniquediffFSP2) > 0:
        print(CRED,BOLD,'Validation Fixed Spaces 2 - Error: n-cols',CEND)
        print(' One or more column dimensions in the user provided fixed space file', BOLD,'is not the same',CEND,' as expected.')
        print(uniquediffFSP)
        display(out)
        print(CRED,BOLD,'Validation Fixed Spaces 2 - Error: k-rows',CEND)
        print(' One or more rows of dimensions in the user provided fixed space file', BOLD,'is not the same',CEND,' as expected.')
        print(uniquediffFSP2)
        display(out)
        raise SystemExit('Validation Fixed Spaces 2 - Errors!')

    if len(uniquediffFSP) == 0 and len(uniquediffFSP2) == 0 and (len(uniquediffFSP) > 0 or len(uniquediffFSP2) > 0):
        print(CGREEN,'Validation Fixed Spaces 2-Passed')


    if len(uniquediffFSP) > 0 and len(uniquediffFSP2) == 0:
        print(CRED,BOLD,'Validation 2 - Error: n-cols',CEND)
        print(' One or more column dimensions in the user provided fixed space file', BOLD,'is not the same',CEND,' as expected.')
        print(uniquediffFSP)
        display(out)
        raise SystemExit('Validation Fixed Spaces 2 - Error: n-cols!')
    else:
        print(CGREEN,'Validation Fixed Spaces 2-Passed: n-cols')

    if len(uniquediffFSP) == 0 and len(uniquediffFSP2) > 0:
        print(CRED,BOLD,'Validation 2 - Error: k-rows',CEND)
        print(' One or more rows of dimensions in the user provided fixed space file', BOLD,'is not the same',CEND,' as expected.')
        print(uniquediffFSP2)
        display(out)
        raise SystemExit('Validation 2 - Error: k-rows!')
    else:
        print(CGREEN,'Validation Fixed Spaces 2-Passed: k-rows',CEND)
        

        
        
##############################################################################
######## sort files - make sure its consistently ordered, normally     #######
######## this is because floorplans are serialized and passed as jsons #######
##############################################################################

# logging_output = output_log(logging_output, "StdOut", "Sorting Four Input Files + Floorplans.")
# logging_output.to_json("/dbfs/" + log_folder + "full_log.json", orient='records')

try:
    ### sort before passing into function
    # sort floors
    floorsDF = sortFloorsDF(rawfloorCapacity)
    # sort Team Size
    sRawTS = sortTeamsDF(rawTS)
    # sort interactions
    rawI = createInteractionsMatrix2(rawIraw)
    sRawI, idxMapI = sortInteractions(rawI)
    # sort distance
    sRawT, idxMapT = sortDistance(rawT)

    # sort floor plan
    #fp_orig, idxMapFPo = sortFloorplan(floorplan_orig)
    # ensure floorplan columns are the same
#     if isinstance(floorplan_modif, pd.DataFrame):
#         floorplan_modif.columns = ['teamName'] + [x.split('_')[1] for x in floorplan_modif.columns if x != 'teamName']
#         fp_modif, idxMapFPi = sortFloorplan(floorplan_modif)


except Exception as e:
#     logging_output = output_log(logging_output, "StdErr", str(e))
#     logging_output.to_json("/dbfs/" + log_folder + "full_log.json", orient='records')
    raise ValueError(e)
    

######              END: local files INPUT 

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
################################################################################################
#                       Variable Workspaces - Helper Functions
################################################################################################

def parseSpaceTypeColumns(floorsDF, idxFirstCol = 1):
    """
    chops first column so that index is floors
    and columns are total capacty, seat type 1, seat type 2, etc.
    
    essentially drops the Floor column and turns it into an index.
    """
    spaceTypesDF = floorsDF.iloc[:,idxFirstCol:]
    spaceTypesDF.index = floorsDF.iloc[:,0]
    spaceTypesDF.index.name = None
    return spaceTypesDF

def parseTeamSeatTypeColumns(sRawTS, idxFirstCol = 1):
    """
    turns column 0 into the index. the index is now Division 1, Division 2, etc
    and the columns are the total team size, role 1, role 2, etc.
    
    essentially drop the Team column in positoin 0 and turn it into an index.
    """
    idxFirstCol = 1
    teamSeatTypesDF = sRawTS.iloc[:,idxFirstCol:]
    teamSeatTypesDF.index = sRawTS.iloc[:,0]
    teamSeatTypesDF.index.name = None
    return teamSeatTypesDF

def createSpaceTypesCatalog(spaceTypesDF):
    """
    NOTE: This function assumes there are only two different space types.
    
    This function splits a floorplan consisting of multiple seat types into two 
    different dataframes specific for the floorplan.
    
    For example, East-Floor1 --> East-Floor1-Offices = 30; East-Floor1-Cubicles = 100
    """
    #seatType1SpacesLabels = []
    #seatType2SpacesLabels = []
    seatType1SpacesSeats = []
    seatType2SpacesSeats = []
    allSpaces = list(spaceTypesDF.index)
    cols = spaceTypesDF.columns
    seatTypeName1 = cols[1]
    seatTypeName2 = cols[2]
    for spaceName in allSpaces:
        seatType1SpacesRow = {}
        newSpaceName1 = spaceName +"_" + seatTypeName1
        seatType1SpacesRow["actual_space_name"] = spaceName
        seatType1SpacesRow["variable_space_name"] = newSpaceName1
        seatType1SpacesRow["seat_type"] = seatTypeName1
        #seatType1SpacesLabels.append(seatType1SpacesRow.copy())
        seatType2SpacesRow = {}
        newSpaceName2 = spaceName +"_" + seatTypeName2
        seatType2SpacesRow["actual_space_name"] = spaceName
        seatType2SpacesRow["variable_space_name"] = newSpaceName2
        seatType2SpacesRow["seat_type"] = seatTypeName2
        #seatType2SpacesLabels.append(seatType2SpacesRow.copy())
        seatsType1Cap = spaceTypesDF.loc[spaceName, cols[1]]
        seatsType2Cap = spaceTypesDF.loc[spaceName, cols[2]]
        seatsTotalCap = seatsType1Cap + seatsType2Cap
        totalRaw = spaceTypesDF.loc[spaceName, cols[0]]
        if (seatsTotalCap != totalRaw):
            print("WARNING: Total seats do not match for Variable Workspace. Space Capacity input file validation ")
            print("total firxt column Total Capacity Column:", totalRaw)
            print("total Seats Type 1:", seatsType1, "Seats Type 2:", seatsType2, "total both columns:", seatsTotal)
        seatType1SpacesRow["space_capacity"] = seatsType1Cap
        seatType2SpacesRow["space_capacity"] = seatsType2Cap
        seatType1SpacesSeats.append(seatType1SpacesRow.copy())
        seatType2SpacesSeats.append(seatType2SpacesRow.copy())
    spaceType1DF = pd.DataFrame(seatType1SpacesSeats)
    spaceType2DF = pd.DataFrame(seatType2SpacesSeats)
    return spaceType1DF, spaceType2DF

def createTeamSeatTypesCatalog(teamSeatTypesDF):
    """
    NOTE: This function assumes there are only two different type of roles.
    
    This function splits a team consisting of multiple roles into two 
    different dataframes specific to the role
    
    For example, Team1 --> Team1-Directors = 10, Team1-Contributors = 100
    """
    seatType1 = []
    seatType2 = []
    allTeams = list(teamSeatTypesDF.index)
    cols = teamSeatTypesDF.columns
    seatTypeName1 = cols[1]
    seatTypeName2 = cols[2]
    for teamName in allTeams:
        seatType1Row = {}
        newTeamName1 = teamName +"_" + seatTypeName1
        seatType1Row["actual_team_name"] = teamName
        seatType1Row["variable_team_name"] = newTeamName1
        seatType1Row["seat_type"] = seatTypeName1
        seatType2Row = {}
        newTeamName2 = teamName +"_" + seatTypeName2
        seatType2Row["actual_team_name"] = teamName
        seatType2Row["variable_team_name"] = newTeamName2
        seatType2Row["seat_type"] = seatTypeName2
        #seatType2SpacesLabels.append(seatType2SpacesRow.copy())
        seatsType1Size = teamSeatTypesDF.loc[teamName, cols[1]]
        seatsType2Size = teamSeatTypesDF.loc[teamName, cols[2]]
        seatsTotalSize = seatsType1Size + seatsType2Size
        totalSizeRaw = teamSeatTypesDF.loc[teamName, cols[0]]
        if (seatsTotalSize != totalSizeRaw):
            print("WARNING: Total seats do not match for Variable Workspace. TEAM SIZE input file validation ")
            print("total first column Size:", totalSizeRaw)
            print("total Seats Type 1:", seatsType1Size, "Seats Type 2:", seatsType2Size, "total both columns:", seatsTotalSize)
        seatType1Row["team_size"] = seatsType1Size
        seatType2Row["team_size"] = seatsType2Size
        seatType1.append(seatType1Row.copy())
        seatType2.append(seatType2Row.copy())
    seatType1DF = pd.DataFrame(seatType1)
    seatType2DF = pd.DataFrame(seatType2)
    return seatType1DF, seatType2DF

##################################################  
# THESE ARE REPEATED FOR N NUMBER OF TEAMS
##################################################
  
def createDummyFloorsDF(spaceTypeCatalog, floorsDF ):
    """
    Create a floorplans for its respective seat type.
    """
    dummyFloorsDF  = spaceTypeCatalog[["variable_space_name", "space_capacity"]].copy()
    dummyFloorsDF.rename(columns = {"variable_space_name" : floorsDF.columns[0], "space_capacity" : floorsDF.columns[1]}, inplace = True)
    return dummyFloorsDF
  
def createDummyTeamSize(teamSeatTypes, sRawTS):
    """
    Create a team size files for its respective role type.
    """
    dummyTS =  teamSeatTypes[["variable_team_name", "team_size"]].copy()
    dummyTS.rename(columns = {"variable_team_name" : sRawTS.columns[0], "team_size":sRawTS.columns[1] }, inplace = True )
    return dummyTS

def createDummyDistanceMatrix(dummyFloorsDF, sRawT):
    """
    Creates a distance file respective to its seat-specific floorplan.
    """
    dummyT = sRawT.copy()
    dummyT.iloc[:,0] = dummyFloorsDF.iloc[:,0]
    firstDummyCol = [dummyT.columns[0]]
    dummyT.columns = firstDummyCol + list(dummyFloorsDF.iloc[:,0])
    return dummyT

def createDummyInteractionsMatrix(dummyTS, sRawI):
    """
    creates an interaction file specific for the role type.
    """
    dummyI = sRawI.copy()
    dummyI.iloc[:,0] = dummyTS.iloc[:,0]
    firstDummyCol = [dummyI.columns[0]]
    dummyI.columns = firstDummyCol + list(dummyTS.iloc[:,0])
    return dummyI

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
### pass information into the rawCsv dictionary for FWAlgo
rawCsv = dict()

rawCsv['distance'] = sRawT
rawCsv['space_capacity'] = floorsDF
rawCsv['interaction'] = sRawI
rawCsv['team_size'] = sRawTS
rawCsv['floorPlan_original'] = fp_orig
rawCsv['floorPlan_modified'] = fp_modif

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
#################################################################################################
########## for JJ - FWOptimizer2 has three new parameters - they are not used in first run ######
########## variableWorkSpaces = False, numSeatTypes = 1, teamSizeVectorOffices = tuple()   ######
########## in addition there is a new constraint - #Constraint 3B. For variable workspaces ######
#################################################################################################


def objectiveFunc(I,X,T):
    return np.trace(T @ X.T @ I @ X)
       #def ObjectiveFunc():
       #    X = np.asmatrix([ [X] * N for _ in range(K)])
       #    ObjFun = ((T % t(X) % I % X).transpose()).trace()
       #    return ObjFun

def boostTeamsInternalCollab(I, booster = 100):
    if (type(I) == np.matrix and I.shape[0] == I.shape[1]):
        boostI = I.copy()
        np.fill_diagonal(boostI, booster*np.max(I))
        return boostI


# def FwOptimizer2(I,X,T,teamSizeVector, floorCapVector, K, N, changeIdx, relConstraints = False, teamIdMap = None, dcDF = None, variableWorkSpaces = False, numSeatTypes = 1, teamSizeVectorOffices = tuple()):
#     """
#     Frank Wolfe algorithm
#     NOTE: Microsoft research paper

#     inputs:
#     ------
#     I: interactions matrix
#     X: floor plan matrix
#     T: distance matrix
#     teamSizeVector: team size file
#     floorCapVector: floor capacity file
#     K: number of teams
#     N: number of floors
#     changeIdx: for interactive mode - delta of user changes - what user changes
#     in interaction mode
#     relConstraints: additional constraints that a user adds (boolean) 
#     teamIdMap: when user uploads the file for the first time, it sorts the file and maps it to an id
#     dcDF: relative constraints input from user for distance constraints
    
#     returns:
#     ------
#     X: floor plan - matrix 
#     success: boolean
#     count: 
#     txx: loss function to minmize
#     """
#     print("This is FW Optimizer with the new Condition as per Aaron Sept 28 ...")
#     #initialize linear search
#     try:
#         eps = 0.001
#         frobeniusNorm = eps+1
#         count = 0
#         gamma = 1.0 / (count + 2)
#         debug = False
#         pacifier = True
#         verbose = False # turn on for debuging
#         success = False
#         initX = X
#         while(frobeniusNorm > eps):
#             ############################################################################################
#             ## Data for Linear Program (formerly LpSove)
#             ############################################################################################
#             #Objective function. Min linear G(Xk)* Y 
#             def getGradient(I, X, T):
#               """

#               """
#               return I @ X @ T + I.T @ X @ T.T 
      
#             Gmat= getGradient(I, X, T)
#             G =  np.array(Gmat.T).ravel()
#             #Constraint 1. Allocate All team members i.e. sum of team = 100% of team size (acroos all floors)
#             teamIdxPerFloor = []
#             for start_i in range(K):
#                 temp_i = [K*j+start_i for j in range(N)]
#                 teamIdxPerFloor.append(temp_i)
#             #Constraint 2. All variables >= 0
#             # x_i allocation >= 0 (see linear program below) 
#             #Constraint 3. Floor <= floor capacity
#             floorIdxAllTeams = []
#             for j in range(0,N*K,K):
#                 temp_j = [j + i for i in range(K)]
#                 floorIdxAllTeams.append(temp_j)
#             #Constraint 4. Interactive 
#             #   changeIdxs = whereChangeFlat(X1,X2, K, N)  #Calculated in driver program
#             if(debug):    
#                 print(" G(Xk) : " , G)
#                 print("Constraint1.X.indexes: ", teamIdxPerFloor)
#                 print("Constraint3.X.indexes: ", floorIdxAllTeams)
#             ############################################################################################
#             ## Linear Program 
#             ############################################################################################
#             #Define problem
#             prob = pulp.LpProblem("FW line search", pulp.LpMinimize )
#             #Define variables
#             var_idxs = list(range(N*K))
#             S = pulp.LpVariable.dict("x_", var_idxs, lowBound=0, upBound = 1)
#             #check_idxs(X, var_idxs)
#             #Define objective Function.
#             prob += sum([G[i] * S[i] for i in range(K*N)])
#             #Constraint 1. Allocate all teams
#             for i in range(K):
#                 prob += sum([S[j] for j in teamIdxPerFloor[i]]) == 1
#             #Constraint 2. All variables >= 0
#             for x_i in range(K*N):
#                 prob += S[x_i] >= 0
                
#             #Constraint 3. floor <= floor capacity
#             teamSize = np.tile(teamSizeVector,N)
#             for j in range(N):
#                  prob +=sum([teamSize[i] * S[i] for i in floorIdxAllTeams[j]])<= floorCapVector[j]
                    
#             ###########################################################################
#             ####                    Here are the changes for the new condition
#             ####                    left many print statements so easier to debug and fine tune in jupyter
#             ####                    those should be removed for last version
#             ###########################################################################                    
            
#             #Constraint 3B. For variable workspaces 
#             print("constraint 3B...")
#             print(variableWorkSpaces, numSeatTypes, teamSizeVectorOffices )
#             isVariableWSpaces = (variableWorkSpaces and numSeatTypes > 1)
#             print("isVariableWSpaces", isVariableWSpaces)
#             if(isVariableWSpaces):
#                 teamSize1 = np.tile(teamSizeVector,N)       #total number of seats requested
#                 teamSize2 = np.tile(teamSizeVectorOffices,N)  #number of office seats requested
#                 print("teamSize1.shape", teamSize1.shape)
#                 print(teamSize1)
#                 print("teamSize2.shape", teamSize2.shape)
#                 print(teamSize2)
#                 for j in range(int(N/numSeatTypes)):
#                     #print(f'Offices: {floorIdxAllTeams[j]}')
#                     #print(f'Offices: {[teamSize2[i] * S[i] for i in floorIdxAllTeams[j]]}')
#                     prob +=sum([teamSize2[i] * S[i] for i in floorIdxAllTeams[j]])<= floorCapVector[j]
#             print("so far so good...")
#             #Constraint 4. Interactive
#             if (changeIdx is not None and len(changeIdx) > 0):
#                 flatX = np.asarray(X).T.ravel()
#                 for i in changeIdx:
#                     prob += S[i] == flatX[i]
#             if(debug):
#                 print("  lpVariable.dict: ",  S)
#                 print("  lpProblem...",prob)
#             #Constraint 5. Relative Constraint
#             isRelCons = (relConstraints and (teamIdMap is not None))
#             if(isRelCons):
#                 prob = addRelConstraintDistanceAvg2Avg(dcDF, T, S, teamIdMap, teamIdxPerFloor, prob)
#                 print("LpProb after adding constraints...")
#                 print(prob)
#             ############################################################################################
#             ## Lp Solve & Wrap up results
#             ############################################################################################
#             print("Solving... ")
#             prob.solve()
#             print("Status:", pulp.LpStatus[prob.status])
#             Xid = { str(v) : k for k,v in S.items()}
#             SoptVector = np.zeros(K*N)
#             for v in prob.variables():
#                 #print(Xid[v.name], v.name, "=", v.varValue )
#                 SoptVector[int(Xid[v.name])] = v.varValue
#             Sopt = np.matrix(SoptVector.reshape(N,K).T)
#             if(debug):
#                 print("result: ", type(Sopt), np.round(Sopt,2))

#             ########################################################################################
#             ### Relative Constraints - Raw Output from Optimizer (for User Testing and Wire-framing)
#             #########################################################################################
#             #TO DO. Add addtional relative constraint test cases
#             if(isRelCons and pulp.LpStatus[prob.status] == 'Optimal'):
#                 success = True
#                 return Sopt, success, count

#             ############################################################################################
#             ## Optimize step size (gamma)
#             ############################################################################################
#             #plugin line search result (1-gamma) Y - gamma X into objective function
#             Y = Sopt
#             tXX = np.trace(T @ X.T @ I @ X)
#             print(f'X: {X}')
#             print(f'tXX: {tXX}')
#             tXY = np.trace(T @ X.T @ I @ Y) + np.trace(T @ Y.T @ I @ X)   
#             tYY = np.trace(T @ Y.T @ I @ Y)
#             a1 = tXX-tXY+tYY # gradient - how big of a jump to take
#             b1 = tXY-2*tXX
#             c1 = tXX
#             #solve for gamma and optimize inside region of interest
#             if(a1 > 0):
#                 #find minimum of quadratic a1*gamma^2 + b1*gamma + c1
#                 z = (-1*b1)/ (2*a1)    
#                 #by definition gamma [0,1] (gamma is weighted average of line between X and Y points in solution space)
#                 if (z>1): gamma = 1
#                 elif (z<0): gamma = 3/(K+2)
#                 else: gamma = z
#             if(a1<0):
#                 #find maximum of quadratic a1*gamma^2 + b1*gamma + c1 
#                 z = (-1*b1)/ (2*a1)
#                 #by definition gamma [0,1] 
#                 if(z>=1): gamma = 3/(K+2)       # follow inverse direction of steepest descent
#                 elif(z<0): gamma = 1
#                 else:                     # inside polygon of valid solution
#                     if((a1+b1)> 0): gamma = 3/(K+2)
#                     else: gamma = 1
#             if (a1==0):     
#                 #find minimum of linear b1*gamma  + c1 sbt [0,1] region
#                 if (b1 >=0): gamma = 3/(K+2)
#                 else: gamma = 1
            

#             #update Xkplus1
#             Xk = X
#             X = Xk + gamma * (Y -Xk)
            
# #             diff_matrix = X.sum(axis=1) - [1]
# #             requested_penalty = np.sum([x**2 for x in np.array(diff_matrix).flatten()]).round(5)
# #             requested_penalty

#             if(debug):
#                 print("tXX :", round(tXX,4)) # what we're trying to minmize (a1)
#                 print("tXY :", round(tXY,4))
#                 print("tYY :", round(tYY,4))
#                 print("a1  : ", round(a1,2))
#                 print("b1  : ", round(b1,2))
#                 print("c1  : ", round(c1,2))
#                 print("z   : ",  (-1*b1)/ (2*a1))
#                 print("gamma: ", gamma)
#             ############################################################################################
#             ## Stop criteria
#             ############################################################################################
#             frobeniusNorm = np.linalg.norm(X-Xk, 'fro')
#             count+=1
#             #emergency break in case not converging
#             #usualy converge after 15 iterations
#             if (count>1000):
#                 print("more than 1000 iterations... cannot converge")
#                 break
#             if(pacifier):
#                 print("lp iteration: " + str(count) + " fNorm : " + str(round(frobeniusNorm,6)))
#         if(verbose):
#                 print("Initial placement... X0")
#                 print(initX)
#                 print("FW Optimal placement... X")
#                 print(X)
#                 print("Last Lp placement... Y")
#                 print(Y)
#         if (frobeniusNorm <= eps):
#             success = True
#             return X, success, count, tXX
#         else:
#             success = False
#             return X, success, count, tXX
#     except:
#         return {"error":"error within FwOptimizer"}
    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
#################################################################################################
########## algoFrankWolfe has three new parameters - they are not used in first run ####
########## variableWorkSpaces = False, numSeatTypes = 1, teamSizeVectorOffices = tuple()   ######
#################################################################################################


#############################################################
# Optimization algorithm - entry point or "driver"
#############################################################
# def algoFrankWolfe(numTries=1, preSetupTeamAllocation = list(), rawCsvs = dict(), runFWOptimizer = True, changeIdx = None, 
#                    isRerun = False, officeFloorCapacityFileFormat = True, 
#                    officeOutput = False,  reDistAdjSeats = False, scaleUnits = False, 
#                    relConstraints = False, fpRank = True, relConstraintsForm = False, runScipyTrOptimizer = False, X0HeurOpt = 2, 
#                    randomSeed=8, variableWorkSpaces=False, numSeatTypes=1, teamSizeVectorOffices = tuple()):
#     """

#     inputs:
#     ------
#     is_rerun: if True then interactive mode; otherwise standard
#     """
#     #select initial placement
#     X0HeuristicsOptions = { 0 : "Azt Classic - WPA Data", 2: "Stacker Tool - Uniform", 3:  "Azt New - Larger  v1" }
#     if(X0HeurOpt not in X0HeuristicsOptions):
#         print("ERROR: " + str(X0HeurOpt) + " Not a valid Heuristics option")
#         print("Please select one of the following Heuristics for initial placement: ", X0HeuristicsOptions)
#     else:
#         print("Using X0Heuristics for initial placement: ", X0HeuristicsOptions[X0HeurOpt])
#     X0Heuristics = X0HeuristicsOptions[X0HeurOpt]
    
    
#     tables = dict()
    
#     rawI = rawCsv["interaction"]
#     rawT = rawCsv["distance"]
#     rawfloorCapacity = rawCsv["space_capacity"]
#     sRawTS = rawCsv["team_size"]
    #originalFloorPlan = rawCsvs["floorPlan_original"]
    #modifiedFloorPlan = rawCsvs["floorPlan_modified"]
#     # getChangeVectors if not Standard
# #     if isRerun:
# #         preSetupTeamAllocation, changeIdx = getChangeVectors(originalFloorPlan, modifiedFloorPlan)


#     #scale units
#     if(scaleUnits == True):
#         zRawT, minMaxRangeT, minT  = minMaxScaler(rawT)
#         print(f'Scaler on: {minMaxRangeT, minT}')
#         print('')
#         #print(zRawT)
#         zRawI, minMaxRangeI, minI  = minMaxScaler(rawI)
#         print(f'Scaler on: {minMaxRangeI, minI}')
#         print('')
#         #print(zRawI)
#     else:
#         zRawT, minMaxRangeT, minT = rawT, 1,0
#         print('Scaler off')
#         #print(zRawT)
#         print('')
#         zRawI, minMaxRangeI, minI = rawI, 1,0
#         #print(zRawI)

#     # Parse relative constraints file
#     isRelCons = ((relConstraints or relConstraintsForm) and (rawCon is not None))
#     if(isRelCons):      
#         runScipyTrOptimizer = True
#         rawDcDF, rawIcDF = parseConstraints(rawCon,sRawTS)
#         print(rawDcDF)
#         dcDF, icDF = preCheckRelConsInput(rawDcDF, rawIcDF)
#     else:
#         runScipyTrOptimizer = False
#         dcDF, icDF = None,None

#     #Team id is a consecutive ID number for identifying teams in the Team Size File (In general, NOT the same number as LP variable dict(x) X_i variable index )
#     teamsIds = getTeamIdsDF(sRawTS)
#     teamIdMap = dict(teamsIds.to_dict(orient = 'split')['data'])
#     teamLabels = dict([ [id, team] for team, id in teamsIds.to_dict(orient = 'split')['data']])
#     #floor id...
#     floorIds = getFloorIdsDF(floorsDF)
#     floorIdMap = dict(floorIds.to_dict(orient = 'split')['data'])
#     floorLabels = dict([ [id, floor] for floor, id in floorIds.to_dict(orient = 'split')['data']])
#     #TO DO... validating team names and team IDs might be something we want to move to general validating routine (i.e. names and IDs need to match across all files, currently assuming sorting names is enough)


#     #fill "tables" for callback
#     tables['teamCollaboration'] = sRawI
#     tables['locationDistance'] = sRawT
#     tables['locationCapacity'] = rawfloorCapacity
#     tables['teamSize'] = sRawTS

#     #get floor names
#     floorNames = getFloorNamesDF(floorsDF)
#     floorColumns = np.delete(zRawT.columns, [0])
    
#     # As a reminder: Distance measured in time units.T[i,j] time to get between floors i and j
#     T = np.asmatrix(zRawT.iloc[0:, 1:])      
#     #N - number of floors
#     N = T.shape[0]

#     if not validateFloors(floorNames, floorColumns, N):
#         error_log = "Floor columns dont match: Check floor capacity vs distance files"
#         return None, None, None, error_log

#     #get teams names
#     teamNames = getTeamNamesDF(sRawTS)
#     rowNames = teamNames
#     teamRowsIdx = np.delete(zRawI.columns, [0])
    
#     I = np.asmatrix(zRawI.iloc[0:, 1:])
#     #K - number of teams
#     K = I.shape[0]
#     validNumTeams =  validateTeams(teamNames, teamRowsIdx, K)
#     if not validNumTeams:
#         error_log = "Team columns dont match: Check team size vs Interactions files"
#         return None, None, None, error_log
#         #print("Team columns dont match: Check team size vs Interactions files")
#         #0/0

#     if(reDistAdjSeats):
#         if(validAdjFile(rawTS)):
#             print("Reading Adj seats columns...")
#             print("Adj Seats Flag: ", reDistAdjSeats )
#             #load adjacency constraints 
#             adjTable, teamSizeDF, adjTeamsDF, adjTeamsList = splitAdjConstraints(sRawTS) 
#             adjSeatsDF = addAdjSeats(teamSizeDF, adjTable)
#             teamSizeFree = tuple(adjSeatsDF["free"])

#             #display adjusted seats (just informative)
#             adjVsAct = adjSeatsDF[list(set(adjSeatsDF.columns) - set(adjSeatsDF.columns[0:3]))].copy()
#             adjVsAct = adjVsAct.astype(int)
#             print(adjVsAct)
#         else:
#             #if user selects the flag but no valid columns available
#             # reverting back 
#             print("Adj seats columns not available...")
#             reDistAdjSeats = False
#             print("Adj Seats Flag : ", reDistAdjSeats )
#             idxTeamSizeCol = 1
#             idxTeamNamesCol = 0
#             adjSeatsDF = sRawTS.iloc[:,idxTeamSizeCol:]
#             adjSeatsDF.index = sRawTS.iloc[:,idxTeamNamesCol]
#     else:
#         # Making Team Size read from FIRST column
#         idxTeamSizeCol = 1
#         idxTeamNamesCol = 0
#         adjSeatsDF = sRawTS.iloc[:,idxTeamSizeCol:]
#         adjSeatsDF.index = sRawTS.iloc[:,idxTeamNamesCol]

#     #capacity and team size to numpy array
#     floorCapVector = floorsDF[floorsDF.columns[1]].values
#     teamSizeVector = getTeamSizeVector2(adjSeatsDF)
#     c = floorCapVector
#     s = np.array(teamSizeVector)
#     print(f'Seats requested: {s}')

#     #some parameters
#     iterFloor = 0         #current floor 
#     count11 = 1
#     print('')
#     smallestObjFun = 1e+29
#     random.seed(randomSeed)  
# #     np.random.seed(randomSeed)
    
   

#     ### running placement
#     for iters in range(numTries):
#         iters = iters + 1
#         print("iteration... ", iters)

#         if (iters <= (N*K)):
#             iterTeam = iters % K       # current team. i.e. teams index for this iteration
#             flag = iters % K           # flag to indicate when all teams have been processed. 
         
#             if (iterTeam == 0): # Line 161 in App.R
#                 # initial placement
#                 iterTeam = K
 
#             try: 
#                 if X0Heuristics == X0HeuristicsOptions[0]:
#                     X0 = pd.DataFrame(arrangeTeams(iterFloor, iterTeam, I, T, floorCapVector, teamSizeVector, False, preSetupTeamAllocation ))
#                     print('Heuristic 0')
#                 elif X0Heuristics == X0HeuristicsOptions[2]:
#                     print('Heuristic 2')
#                     # first seed will use the basic team placement
#                     if randomSeed == 0:
#                         flatX0 = True
#                     # the rest will be random
#                     else:
#                         flatX0 = False
#                     X0 = pd.DataFrame(arrangeTeams2(iterFloor, iterTeam, I, T, K, N, floorCapVector, teamSizeVector, False, preSetupTeamAllocation, useTeamRatio = False, flatX0 = flatX0 ))
      
#                 # running FW algo to produce Space Planning Matrix
#                 teamSizeMat = np.repeat(teamSizeVector,N).reshape(K,N)
#                 tempRawX0 = X0
#                 X0 = np.round(tempRawX0 / teamSizeMat,4)
#                 X = np.asmatrix(X0.values)


#                 Xopt, FWoptStatus, FWOptCount, tXX = FwOptimizer2(I,X,T,teamSizeVector, floorCapVector, K, N, changeIdx, 
#                                                                   relConstraints = relConstraints, teamIdMap = teamIdMap, dcDF = dcDF, 
#                                                                   variableWorkSpaces=variableWorkSpaces, numSeatTypes=numSeatTypes, 
#                                                                   teamSizeVectorOffices = teamSizeVectorOffices)
                  

#                 XoptDF = pd.DataFrame(np.round(np.multiply(Xopt, teamSizeMat)))
#                 if XoptDF.shape[1] == floorColumns.size:
#                     XoptDF.columns = floorColumns
#                     XoptDF.index = list(rowNames["Team"].values)
#                     XoptDF.columns = XoptDF.columns.str.replace(',',' -')
#                     if (reDistAdjSeats):
#                         newFP = reDistributeAdjSeats(XoptDF, adjTeamsDF, adjTeamsList)
#                     else:
#                         newFP = XoptDF
#                     if (officeOutput):
#                         Xoutput, floorsDF2 = splitByOfficeWorkCapacity(newFP, floorsDF)
#                     else:
#                         Xoutput = newFP

#                     tables['floorPlan'] = Xoutput.to_dict(orient='dict')
#                     tables['teamCollaboration'] = tables['teamCollaboration'].set_index(tables['teamCollaboration'].columns[0]).to_dict(orient='dict')
#                     tables['locationDistance'] = tables['locationDistance'].set_index(tables['locationDistance'].columns[0]).to_dict(orient='dict')
#                     tables['locationCapacity'] = tables['locationCapacity'].set_index(tables['locationCapacity'].columns[0]).to_dict(orient='dict')
#                     tables['teamSize'] = tables['teamSize'].set_index(tables['teamSize'].columns[0]).to_dict(orient='dict')
#                     #print(FWoptStatus, FWOptCount, tXX)
#                 return tables, FWoptStatus, FWOptCount, tXX
#             except Exception as e:
#                 print(e)
#                 print("error generating the floorplan")

def algoFrankWolfe(numTries=1, preSetupTeamAllocation = list(), rawCsvs = dict(), runFWOptimizer = True, changeIdx = None, isRerun = False, officeFloorCapacityFileFormat = True, officeOutput = False,  reDistAdjSeats = False, scaleUnits = False, relConstraints = False, fpRank = True, relConstraintsForm = False, runScipyTrOptimizer = False, X0HeurOpt = 2, randomSeed=8):
    """

    inputs:
    ------
    is_rerun: if True then interactive mode; otherwise standard
    """
    #select initial placement
    X0HeuristicsOptions = { 0 : "Azt Classic - WPA Data", 2: "Stacker Tool - Uniform", 3:  "Azt New - Larger  v1" }
    if(X0HeurOpt not in X0HeuristicsOptions):
        print("ERROR: " + str(X0HeurOpt) + " Not a valid Heuristics option")
        print("Please select one of the following Heuristics for initial placement: ", X0HeuristicsOptions)
    else:
        print("Using X0Heuristics for initial placement: ", X0HeuristicsOptions[X0HeurOpt])
    X0Heuristics = X0HeuristicsOptions[X0HeurOpt]
    
    
    tables = dict()
    
    rawI = rawCsvs["interaction"]
    rawT = rawCsvs["distance"]
    rawfloorCapacity = rawCsvs["space_capacity"]
    sRawTS = rawCsvs["team_size"]
    originalFloorPlan = rawCsvs["floorPlan_original"]
    modifiedFloorPlan = rawCsvs["floorPlan_modified"]
    # getChangeVectors if not Standard
    if isRerun:
      preSetupTeamAllocation, changeIdx = getChangeVectors(originalFloorPlan, modifiedFloorPlan)


    #scale units
    if(scaleUnits == True):
        zRawT, minMaxRangeT, minT  = minMaxScaler(rawT)
        zRawI, minMaxRangeI, minI  = minMaxScaler(rawI)
    else:
        zRawT, minMaxRangeT, minT = rawT, 1,0
        zRawI, minMaxRangeI, minI = rawI, 1,0

    # Parse relative constraints file
    isRelCons = ((relConstraints or relConstraintsForm) and (rawCon is not None))
    if(isRelCons):      
        runScipyTrOptimizer = True
        rawDcDF, rawIcDF = parseConstraints(rawCon,sRawTS)
        print(rawDcDF)
        dcDF, icDF = preCheckRelConsInput(rawDcDF, rawIcDF)
    else:
        runScipyTrOptimizer = False
        dcDF, icDF = None,None

    #Team id is a consecutive ID number for identifying teams in the Team Size File (In general, NOT the same number as LP variable dict(x) X_i variable index )
    teamsIds = getTeamIdsDF(sRawTS)
    teamIdMap = dict(teamsIds.to_dict(orient = 'split')['data'])
    teamLabels = dict([ [id, team] for team, id in teamsIds.to_dict(orient = 'split')['data']])
    #floor id...
    floorIds = getFloorIdsDF(floorsDF)
    floorIdMap = dict(floorIds.to_dict(orient = 'split')['data'])
    floorLabels = dict([ [id, floor] for floor, id in floorIds.to_dict(orient = 'split')['data']])
    #TO DO... validating team names and team IDs might be something we want to move to general validating routine (i.e. names and IDs need to match across all files, currently assuming sorting names is enough)


    #fill "tables" for callback
    tables['teamCollaboration'] = sRawI
    tables['locationDistance'] = sRawT
    tables['locationCapacity'] = rawfloorCapacity
    tables['teamSize'] = sRawTS

    #get floor names
    floorNames = getFloorNamesDF(floorsDF)
    floorColumns = np.delete(zRawT.columns, [0])
    
    # As a reminder: Distance measured in time units.T[i,j] time to get between floors i and j
    T = np.asmatrix(zRawT.iloc[0:, 1:])      
    #N - number of floors
    N = T.shape[0]

    if not validateFloors(floorNames, floorColumns, N):
        error_log = "Floor columns dont match: Check floor capacity vs distance files"
        print(error_log)
        return None, None, None, error_log


    #get teams names
    teamNames = getTeamNamesDF(sRawTS)
    rowNames = teamNames
    teamRowsIdx = np.delete(zRawI.columns, [0])
    
    I = np.asmatrix(zRawI.iloc[0:, 1:])
    #K - number of teams
    K = I.shape[0]
    validNumTeams =  validateTeams(teamNames, teamRowsIdx, K)
    if not validNumTeams:
        error_log = "Team columns dont match: Check team size vs Interactions files"
        print(error_log)
        return None, None, None, error_log

    if(reDistAdjSeats):
        if(validAdjFile(rawTS)):
            print("Reading Adj seats columns...")
            print("Adj Seats Flag: ", reDistAdjSeats )
            #load adjacency constraints 
            adjTable, teamSizeDF, adjTeamsDF, adjTeamsList = splitAdjConstraints(sRawTS) 
            adjSeatsDF = addAdjSeats(teamSizeDF, adjTable)
            teamSizeFree = tuple(adjSeatsDF["free"])

            #display adjusted seats (just informative)
            adjVsAct = adjSeatsDF[list(set(adjSeatsDF.columns) - set(adjSeatsDF.columns[0:3]))].copy()
            adjVsAct = adjVsAct.astype(int)
            print(adjVsAct)
        else:
            #if user selects the flag but no valid columns available
            # reverting back 
            print("Adj seats columns not available...")
            reDistAdjSeats = False
            print("Adj Seats Flag : ", reDistAdjSeats )
            idxTeamSizeCol = 1
            idxTeamNamesCol = 0
            adjSeatsDF = sRawTS.iloc[:,idxTeamSizeCol:]
            adjSeatsDF.index = sRawTS.iloc[:,idxTeamNamesCol]
    else:
        # Making Team Size read from FIRST column
        idxTeamSizeCol = 1
        idxTeamNamesCol = 0
        adjSeatsDF = sRawTS.iloc[:,idxTeamSizeCol:]
        adjSeatsDF.index = sRawTS.iloc[:,idxTeamNamesCol]

    #capacity and team size to numpy array
    floorCapVector = floorsDF[floorsDF.columns[1]].values
    teamSizeVector = getTeamSizeVector2(adjSeatsDF)
    c = floorCapVector
    s = np.array(teamSizeVector)

    #some parameters
    iterFloor = 0         #current floor 
    count11 = 1
    print('')
    smallestObjFun = 1e+29
    random.seed(randomSeed)  
#     np.random.seed(randomSeed)
    
   

    ### running placement
    for iters in range(numTries):
        iters = iters + 1
        print("iteration... ", iters)

        if (iters <= (N*K)):
            iterTeam = iters % K       # current team. i.e. teams index for this iteration
            flag = iters % K           # flag to indicate when all teams have been processed. 
         
            if (iterTeam == 0): # Line 161 in App.R
                # initial placement
                iterTeam = K
 
            try: 
                if X0Heuristics == X0HeuristicsOptions[0]:
                    X0 = pd.DataFrame(arrangeTeams(iterFloor, iterTeam, I, T, floorCapVector, teamSizeVector, False, preSetupTeamAllocation ))
                    print('Heuristic 0')
                elif X0Heuristics == X0HeuristicsOptions[2]:
                    print('Heuristic 2')
                    # first seed will use the basic team placement
                    if randomSeed == 0:
                      flatX0 = True
                    # the rest will be random
                    else:
                      flatX0 = False
                    X0 = pd.DataFrame(arrangeTeams2(iterFloor, iterTeam, I, T, K, N, floorCapVector, teamSizeVector, False, preSetupTeamAllocation, useTeamRatio = False, flatX0 = flatX0 ))
      
                # running FW algo to produce Space Planning Matrix
                teamSizeMat = np.repeat(teamSizeVector,N).reshape(K,N)
                tempRawX0 = X0
                X0 = np.round(tempRawX0 / teamSizeMat,4)
                X = np.asmatrix(X0.values)
                
                Xopt, FWoptStatus, FWOptCount, tXX = FwOptimizer2(I,X,T,teamSizeVector, floorCapVector, K, N, changeIdx, relConstraints = relConstraints, teamIdMap = teamIdMap, dcDF = dcDF)
               
                print(Xopt)
#                 Xopt = np.asarray(Xopt, dtype= 'float64')
                XoptDF = pd.DataFrame(np.round(np.multiply(Xopt, teamSizeMat)))
                if XoptDF.shape[1] == floorColumns.size:
                    XoptDF.columns = floorColumns
                    XoptDF.index = list(rowNames["Team"].values)
                    XoptDF.columns = XoptDF.columns.str.replace(',',' -')
                    if (reDistAdjSeats):
                        newFP = reDistributeAdjSeats(XoptDF, adjTeamsDF, adjTeamsList)
                    else:
                        newFP = XoptDF
                    if (officeOutput):
                        Xoutput, floorsDF2 = splitByOfficeWorkCapacity(newFP, floorsDF)
                    else:
                        Xoutput = newFP

                    tables['floorPlan'] = Xoutput.to_dict(orient='dict')
                    tables['teamCollaboration'] = tables['teamCollaboration'].set_index(tables['teamCollaboration'].columns[0]).to_dict(orient='dict')
                    tables['locationDistance'] = tables['locationDistance'].set_index(tables['locationDistance'].columns[0]).to_dict(orient='dict')
                    tables['locationCapacity'] = tables['locationCapacity'].set_index(tables['locationCapacity'].columns[0]).to_dict(orient='dict')
                    tables['teamSize'] = tables['teamSize'].set_index(tables['teamSize'].columns[0]).to_dict(orient='dict')
                    #print(FWoptStatus, FWOptCount, tXX)
                return tables, FWoptStatus, FWOptCount, tXX
            except Exception as e:
                print(e)
                print("error generating the floorplan")

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
# manually set
X0HeurOpt = 2
rawCsvs = rawCsv

# we run the algorithm 13 times using random seeds to make sure we get the best floor plan
np.random.seed(42)
randomSeeds = [0, 7, 8, 10, 14, 42] + [np.random.randint(10000) for x in range(7)]


#################################################################################################
########## Running the algorithm  - can do it for any mode                     ##################
#################################################################################################

try:
    # manually set
    X0HeurOpt = 2
    # parsed from dcitionary
    numTries = mode_params['numTries']
    preSetupTeamAllocation = mode_params['preSetupTeamAllocation']
    rawCsvs = rawCsv
    runFWOptimizer = mode_params['runFWOptimizer']
    changeIdx = mode_params['changeIdx']
    #flatInteractionBool = mode_params['flatInteractionBool']
    isRerun = mode_params['isRerun']
    officeFloorCapacityFileFormat = mode_params['officeFloorCapacityFileFormat']
    officeOutput = mode_params['officeOutput']
    reDistAdjSeats = mode_params['reDistAdjSeats']
    scaleUnits = mode_params['scaleUnits']
    relConstraints = mode_params['relConstraints']
    fpRank = mode_params['fpRank']
    relConstraintsForm = mode_params['relConstraintsForm']
    runScipyTrOptimizer =  mode_params['runScipyTrOptimizer']
    
    # RUNNING THE ALGORITHM
    np.random.seed(42)
    randomSeeds = [0, 7, 8, 10, 14, 42] + [np.random.randint(10000) for x in range(7)]
    tables_list = [] # for tracking all floorplans for the 13 rounds
    scores = [] # keep track of tXX scores
    
    for seed in randomSeeds:
        tables, FWoptStatus, FWoptCount, tXX = algoFrankWolfe(numTries, preSetupTeamAllocation, rawCsvs, runFWOptimizer, changeIdx, 
                                                              isRerun, officeFloorCapacityFileFormat, officeOutput,  reDistAdjSeats, scaleUnits, 
                                                              relConstraints, fpRank, relConstraintsForm, 
                                                              runScipyTrOptimizer, X0HeurOpt, randomSeed=seed)
#         tables, FWoptStatus, FWoptCount, tXX = algoFrankWolfe(numTries, preSetupTeamAllocation, rawCsvs, 
#                                                               runFWOptimizer, changeIdx, isRerun, officeFloorCapacityFileFormat,
#                                                               officeOutput, reDistAdjSeats, scaleUnits, 
#                                                               relConstraints, fpRank, relConstraintsForm, 
#                                                               runScipyTrOptimizer, X0HeurOpt, randomSeed=seed,
#                                                              variableWorkSpaces=False, numSeatTypes=1, teamSizeVectorOffices = tuple())
        tables_list.append(tables)
        scores.append(np.round(tXX, 2))
  
    # choose best table
    best_score_indx = np.argmin(scores) # save this seed to history map (so if you run it again you same seed) 
    best_floor_plan = tables_list[best_score_indx]
    print('\n\n\n')
    print(f'Status: {FWoptStatus}')
    print(scores)
    print(f'Best iteration: {best_score_indx}. Best Score: {scores[best_score_indx]}')

except Exception as e:
    raise ValueError(e)
    

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
   

In [ ]:
savefinsh = '<div class="alert alert-block alert-success"><b> Done: </b> Your ' + '<b>'+out_filename+'</b> file has successfully saved back to a app_results folder inside the location of this Notebook.</div>'


if variableSeats:
    initalDF = pd.DataFrame(best_floor_plan['floorPlan'])
    
else:
    #####################
    ### Save results  ###
    #####################
    floorplan_df = pd.DataFrame(best_floor_plan['floorPlan'])
    floorplan_final_path = os.getcwd() + '\\'+ 'app_results'+'\\'+out_filename
    floorplan_df.to_excel(r''+ floorplan_final_path, sheet_name='Floorplan')
    
    print(floorplan_final_path)
    display(Markdown(savefinsh))
    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
   


In [ ]:
# #################################################################################################
# ########## Pre-process files again to prepare for Variable Seats  ######################
# #################################################################################################

# clear_output()

# if variableSeats:
#     # GET EMPTY FLOOR PLAN
#     T = np.asmatrix(sRawT.iloc[0:, 1:]) 
#     #N - number of floors
#     N = T.shape[0]

#     I = np.asmatrix(sRawI.iloc[0:, 1:])
#     #K - number of teams
#     K = I.shape[0]
#     originalFloorPlan = np.zeros((K,N))
#     print(f'original floor plan (empty) shape: {originalFloorPlan.shape}')

#     # TREAT THIS AS MODIFIED - preSetupTeamAllocation will be passe din second go
#     Xoutput2 = pd.DataFrame(tables['floorPlan'])
#     modifiedFloorPlan = Xoutput2
#     preSetupTeamAllocation, changeIdx = getChangeVectors(originalFloorPlan, modifiedFloorPlan)
#     changeIdx = []

#     ############################################################
#     ###### PREPROCESSING AGAIN
#     ############################################################

#     # CREATE COMBINED FLOORPLAN
#     floorNamesSeatType2 = getFloorNamesDF(dummyFloorsDF2)
#     floorColumnsSeatType2 = np.delete(dummyT2.columns, [0])

#     dummyFloorplanSeatType2 = pd.DataFrame(np.zeros((K,N)))
#     dummyFloorplanSeatType2.columns = floorColumnsSeatType2

#     Xdummy1 = dummyFloorplanSeatType2
#     Xdummy1.index = origTS.iloc[:,0]

#     Xdummy2 = Xoutput2.copy()
#     Xdummy2.index = origTS.iloc[:,0]

#     variableSpacesFloorplan = pd.concat([Xdummy2,Xdummy1], axis = 1)

#     # CREATE COMBINED DISTANCE
#     #create two copies and concatenate horizontally - for seat Type 1
#     seatType1Names  = dummyT1.iloc[:,0]
#     tempDummyT2 = dummyT2.copy()
#     tempDummyT2.iloc[:,0] = seatType1Names
#     dummyT1T2repHorizontal1 = pd.merge(dummyT1,tempDummyT2, left_on =  [dummyT1.columns[0]], right_on =  [tempDummyT2.columns[0]])
#     #print(dummyT1.shape, dummyT2.shape, dummyT1T2repHorizontal1.shape)
#     dummyT1T2repHorizontal1Cols = dummyT1T2repHorizontal1.columns

#     #create two copies and concatenate horizontally - for seat Type 2
#     seatType2Names  = dummyT2.iloc[:,0]
#     tempDummyT1 = dummyT1.copy()
#     tempDummyT1.iloc[:,0] = seatType2Names
#     dummyT2T1repHorizontal2 = pd.merge(dummyT2,tempDummyT1, left_on =  [dummyT2.columns[0]], right_on =  [tempDummyT1.columns[0]])
#     #print(dummyT1.shape, dummyT2.shape, dummyT2T1repHorizontal2.shape)
#     #print(dummyT2T1repHorizontal2.columns)
#     dummyT2T1repHorizontal2.columns = dummyT1T2repHorizontal1Cols

#     dummyT1T2Combined = pd.concat([dummyT1T2repHorizontal1, dummyT2T1repHorizontal2], axis = 0, ignore_index = True)

#     # CREATE COMBINED FLOOR CAPACITY
#     dummyFloorsDF12Combined = pd.concat([dummyFloorsDF1, dummyFloorsDF2], axis = 0, ignore_index = True)


#     sRawI = origI
#     sRawT = dummyT1T2Combined
#     floorsDF = dummyFloorsDF12Combined
#     sRawTS = origTS

#     ### pass into dictionary
#     rawCsv['distance'] = sRawT
#     rawCsv['space_capacity'] = floorsDF
#     rawCsv['interaction'] = sRawI
#     rawCsv['team_size'] = sRawTS
    

# display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:
# #######################################################################################
# ########## Running FW algo a second time - with variable seats parameters   ###########
# ##########                       turned on                                  ###########
# #######################################################################################
# clear_output()

# if variableSeats:
#     #for Variable Workspaces - turn these flags on
#     variableWorkSpaces = True  # flag to indicate Aaron Constraint Sept 28
#     numSeatTypes = 2 
#     teamSizeVectorOffices = getTeamSizeVector(dummyTS1)

#     # now run WSP again
#     try:
#         tables_list = []
#         scores = []

#         #start_time = time.time()
#         for seed in randomSeeds:
#             tables, FWoptStatus, FWoptCount, tXX = algoFrankWolfe(numTries, preSetupTeamAllocation, rawCsvs, 
#                                                               runFWOptimizer, changeIdx, isRerun, officeFloorCapacityFileFormat,
#                                                               officeOutput, reDistAdjSeats, scaleUnits, 
#                                                               relConstraints, fpRank, relConstraintsForm, 
#                                                               runScipyTrOptimizer, X0HeurOpt, randomSeed=seed,
#                                                              variableWorkSpaces=True, numSeatTypes=2, teamSizeVectorOffices = teamSizeVectorOffices)

#             # keep track of floorplans, keep track of scores
#             tables_list.append(tables)
#             scores.append(np.round(tXX, 5))
            
            
#         # choose best table
#         best_score_indx = np.argmin(scores) # save this seed to history map (so if you run it again you same seed) 
#         best_floor_plan = tables_list[best_score_indx]
#         print('\n\n\n')
#         print(f'Status: {FWoptStatus}')
#         print(scores)
#         print(f'Best iteration: {best_score_indx}. Best Score: {scores[best_score_indx]}')
#     except Exception as e:
#         raise ValueError(e)
    
#     final_floorplan_df = pd.DataFrame(best_floor_plan['floorPlan'])
    
# display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
   

In [ ]:
# clear_output()
# if variableSeats:
#     #####################
#     ### Save results  ###
#     #####################
#     final_floorplan_df = pd.DataFrame(best_floor_plan['floorPlan'])
#     floorplan_final_path = os.getcwd() + '\\'+ 'app_results'+'\\'+out_filename
#     final_floorplan_df.to_excel(r''+ floorplan_final_path, sheet_name='Floorplan')
    
#     print(floorplan_final_path)
#     display(Markdown(savefinsh))
    